<a href="https://colab.research.google.com/github/nao329/accelerationOpticalFlow/blob/sub/accelerationOpticalFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pwd
%cd /content/drive/MyDrive/創成課題/第4回/

### オプティカルフロー

- 動画ファイル
  *   uniformMotionWithNoEdit
  *   fall_ball
  *   single_loop_ball
  *   bound_ball_with_damp
  *   bound_ball_mov





### オプティカルフロー関数の実装

In [ ]:
import cv2

def calcOpticalFlowFarneback(prev, next, flow=None, pyr_scale=0.5, levels=3, winsize=15,
                             iterations=3, poly_n=5, poly_sigma=1.2, flags=0):
    """
    Farneback法を使用してオプティカルフローを計算します。

    引数:
        prev (ndarray): 前のフレーム（グレースケール画像）。
        next (ndarray): 次のフレーム（グレースケール画像）。
        flow (ndarray, optional): 初期フロー（必要に応じて）。デフォルトはNone。
        pyr_scale (float, optional): 各ピラミッドレベルでの画像スケール。デフォルトは0.5。
        levels (int, optional): ピラミッドレベルの数。デフォルトは3。
        winsize (int, optional): 窓サイズ。デフォルトは15。
        iterations (int, optional): 各レベルでの反復回数。デフォルトは3。
        poly_n (int, optional): 多項式展開のピクセル近傍サイズ。デフォルトは5。
        poly_sigma (float, optional): Gaussian標準偏差。デフォルトは1.2。
        flags (int, optional): オプションフラグ。デフォルトは0。

    戻り値:
        flow (ndarray): 計算されたオプティカルフロー。
    """
    # OpenCVのcalcOpticalFlowFarneback関数を使用してフローを計算
    flow = cv2.calcOpticalFlowFarneback(
        prev, next, flow, pyr_scale, levels, winsize,
        iterations, poly_n, poly_sigma, flags
    )
    return flow


In [ ]:
import cv2
import numpy as np

def calc_farneback_optical_flow(prev0, next0, flow0=None, pyr_scale=0.5, num_levels=5, flags=0,
                                win_size=15, num_iters=3, poly_n=5, poly_sigma=1.2):
    """
    Farneback法を使用して密なオプティカルフローを計算します。

    引数:
        prev0 (numpy.ndarray): 前のフレーム（グレースケール画像）。
        next0 (numpy.ndarray): 次のフレーム（グレースケール画像）。
        flow0 (numpy.ndarray, optional): 初期フローまたは出力フロー。デフォルトはNone。
        pyr_scale (float): 各ピラミッドレベルでの画像スケール（1未満）。
        num_levels (int): ピラミッドレベルの数。
        flags (int): アルゴリズムのフラグ。
        win_size (int): 平均化に使用されるウィンドウサイズ。
        num_iters (int): 各ピラミッドレベルでの反復回数。
        poly_n (int): 多項式展開の近傍サイズ。
        poly_sigma (float): ガウス分布の標準偏差。

    戻り値:
        numpy.ndarray: 計算されたフロー（形状 (h, w, 2) のnumpy配列）。
    """

    min_size = 32
    img = [prev0, next0]

    # 入力の検証
    assert prev0.shape == next0.shape and len(prev0.shape) == 2, "入力画像は同じサイズのグレースケール画像である必要があります。"
    assert pyr_scale < 1, "pyr_scaleは1未満でなければなりません。"

    if flags & cv2.OPTFLOW_USE_INITIAL_FLOW:
        assert flow0 is not None, "OPTFLOW_USE_INITIAL_FLOWフラグが設定されている場合、初期フローを提供する必要があります。"
        assert flow0.shape[:2] == prev0.shape[:2] and flow0.shape[2] == 2 and flow0.dtype == np.float32, "flow0の次元が正しくありません。"
    else:
        flow0 = np.zeros((prev0.shape[0], prev0.shape[1], 2), dtype=np.float32)

    prevFlow = None
    levels = num_levels

    # 画像サイズとmin_sizeに基づいてレベル数を調整
    scale = 1.0
    for k in range(levels):
        scale *= pyr_scale
        if prev0.shape[1]*scale < min_size or prev0.shape[0]*scale < min_size:
            break
    levels = k

    for k in range(levels, -1, -1):
        # このレベルのスケールを計算
        scale = pyr_scale ** k

        sigma = (1.0 / scale - 1) * 0.5
        smooth_sz = int(round(sigma * 5)) | 1
        smooth_sz = max(smooth_sz, 3)

        width = int(round(prev0.shape[1] * scale))
        height = int(round(prev0.shape[0] * scale))

        if k > 0:
            flow = np.zeros((height, width, 2), dtype=np.float32)
        else:
            flow = flow0

        if prevFlow is None:
            if flags & cv2.OPTFLOW_USE_INITIAL_FLOW:
                flow = cv2.resize(flow0, (width, height), interpolation=cv2.INTER_AREA)
                flow *= scale
            else:
                flow = np.zeros((height, width, 2), dtype=np.float32)
        else:
            flow = cv2.resize(prevFlow, (width, height), interpolation=cv2.INTER_LINEAR)
            flow *= 1.0 / pyr_scale

        # このピラミッドレベルの画像を準備
        R = [None, None]
        for i in range(2):
            fimg = img[i].astype(np.float32)
            fimg = cv2.GaussianBlur(fimg, (smooth_sz, smooth_sz), sigma)
            I = cv2.resize(fimg, (width, height), interpolation=cv2.INTER_LINEAR)
            # Farnebackの多項式展開を行う（内部関数のため直接使用不可）
            R[i] = farneback_poly_exp(I, poly_n, poly_sigma)

        # フローマトリックスの計算
        M = farneback_update_matrices(R[0], R[1], flow)

        # フローを反復的に更新
        for i in range(num_iters):
            if flags & cv2.OPTFLOW_FARNEBACK_GAUSSIAN:
                flow = farneback_update_flow_gaussian(R[0], R[1], flow, M, win_size, i < num_iters - 1)
            else:
                flow = farneback_update_flow(R[0], R[1], flow, M, win_size, i < num_iters - 1)

        prevFlow = flow.copy()

    return flow

def farneback_poly_exp(I, poly_n, poly_sigma):
    """
    画像の多項式展開を計算します。

    OpenCVのFarneback実装の内部関数であり、Pythonでは直接アクセスできません。
    この関数を再実装するには高度な数学的知識が必要です。

    引数:
        I (numpy.ndarray): 現在のピラミッドレベルの入力画像。
        poly_n (int): ピクセル近傍のサイズ。
        poly_sigma (float): ガウシアンの標準偏差。

    戻り値:
        numpy.ndarray: 画像の多項式展開結果。
    """
    # この部分は高度な実装が必要なため、ここでは擬似的に入力をそのまま返します。
    # 実際のアプリケーションでは、この部分の正確な実装が必要です。
    return I  # 擬似的な戻り値

def farneback_update_matrices(R0, R1, flow):
    """
    Farnebackアルゴリズムで使用されるフローマトリックスを計算します。

    引数:
        R0 (numpy.ndarray): 最初の画像の多項式展開。
        R1 (numpy.ndarray): 2番目の画像の多項式展開。
        flow (numpy.ndarray): 現在のフロー推定。

    戻り値:
        numpy.ndarray: フローマトリックス。
    """
    # この部分も高度な実装が必要です。
    M = np.zeros_like(flow)  # 擬似的な戻り値
    return M

def farneback_update_flow(R0, R1, flow, M, win_size, update_matrices):
    """
    ボックスフィルターを使用してフローを更新します。

    引数:
        R0, R1, flow, M: 前述の通り。
        win_size (int): ウィンドウサイズ。
        update_matrices (bool): マトリックスを更新するかどうか。

    戻り値:
        numpy.ndarray: 更新されたフロー。
    """
    # この部分も高度な実装が必要です。
    return flow  # 擬似的な戻り値

def farneback_update_flow_gaussian(R0, R1, flow, M, win_size, update_matrices):
    """
    ガウシアンブラーを使用してフローを更新します。

    引数:
        R0, R1, flow, M: 前述の通り。
        win_size (int): ウィンドウサイズ。
        update_matrices (bool): マトリックスを更新するかどうか。

    戻り値:
        numpy.ndarray: 更新されたフロー。
    """
    # この部分も高度な実装が必要です。
    return flow  # 擬似的な戻り値


### farneback法のスクラッチ

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# 動画のキャプチャを開始
cap = cv2.VideoCapture('uniformMotionWithNoEdit.mp4')

# ランダムな色を生成
color = np.random.randint(0, 255, (100, 3))

# 最初のフレームを読み込み、グレースケールに変換
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# 描画用のマスクを作成
mask = np.zeros_like(old_frame)

def calc_optical_flow_farneback_scratch(old_gray, frame_gray, num_iterations=3, window_size=15, poly_n=5, poly_sigma=1.2):
    h, w = old_gray.shape
    flow = np.zeros((h, w, 2), dtype=np.float64)

    # ピラミッド構造を作成し、段階的に追跡を行う
    for iteration in range(num_iterations):
        for y in range(0, h - window_size, window_size):
            for x in range(0, w - window_size, window_size):
                # 窓を切り出して、それぞれの差分を取る
                old_patch = old_gray[y:y + window_size, x:x + window_size]
                new_patch = frame_gray[y:y + window_size, x:x + window_size]

                # グラディエント計算（画像の勾配を計算して物体の動きを推定する）
                grad_x = cv2.Sobel(old_patch, cv2.CV_64F, 1, 0, ksize=poly_n)
                grad_y = cv2.Sobel(old_patch, cv2.CV_64F, 0, 1, ksize=poly_n)
                grad_t = (new_patch.astype(np.float64) - old_patch.astype(np.float64))

                # 最小二乗法で移動ベクトルを計算
                A = np.stack((grad_x.ravel(), grad_y.ravel()), axis=1)
                b = -grad_t.ravel()

                # 方程式を解く（最小二乗解）
                nu = np.linalg.lstsq(A, b, rcond=None)[0]
                flow[y:y + window_size, x:x + window_size, 0] += nu[0]
                flow[y:y + window_size, x:x + window_size, 1] += nu[1]

    # ガウシアンフィルタで平滑化
    flow[:, :, 0] = cv2.GaussianBlur(flow[:, :, 0], (poly_n, poly_n), poly_sigma)
    flow[:, :, 1] = cv2.GaussianBlur(flow[:, :, 1], (poly_n, poly_n), poly_sigma)

    return flow

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 現在のフレームをグレースケールに変換
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # フルスクラッチでFarneback法に基づくオプティカルフローを計算
    flow = calc_optical_flow_farneback_scratch(old_gray, frame_gray, num_iterations=5, window_size=21, poly_n=7, poly_sigma=1.5)

    # フローのベクトルを描画
    h, w = frame_gray.shape[:2]
    step = 8  # より小さなステップでサンプリングして精度を向上させる
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx * 100, y + fy * 100]).T.reshape(-1, 2, 2)
    lines = np.int32(lines)
    for (x1, y1), (x2, y2) in lines:
        # mask = cv2.line(mask, (x1, y1), (x2, y2), color[np.random.randint(0, len(color))].tolist(), 1)
        # frame = cv2.circle(frame, (x1, y1), 1, color[np.random.randint(0, len(color))].tolist(), -1)
        mask = cv2.line(mask, (x1, y1), (x2, y2), (128, 128, 128), 1)
        frame = cv2.circle(frame, (x1, y1), 1, (128, 128, 128), -1)

    img = cv2.add(frame, mask)

    # 結果を表示
    print('Farneback Optical Flow - High Precision (Scratch)')
    cv2_imshow(img)


    # 次のフレームの準備
    old_gray = frame_gray.copy()

# リリースとウィンドウの破棄
cap.release()


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# 動画のキャプチャを開始
cap = cv2.VideoCapture('fall_ball.mp4')

# ランダムな色を生成
color = np.random.randint(0, 255, (100, 3))

# 最初のフレームを読み込み、グレースケールに変換
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# 描画用のマスクを作成
mask = np.zeros_like(old_frame)

def calc_optical_flow_farneback_scratch_with_pyramid(old_gray, frame_gray, levels=3, num_iterations=3, window_size=15, poly_n=5, poly_sigma=1.2):
    h, w = old_gray.shape
    flow = np.zeros((h, w, 2), dtype=np.float32)

    # ピラミッド構造を作成
    pyramids_old = [old_gray]
    pyramids_new = [frame_gray]
    for level in range(1, levels):
        pyramids_old.append(cv2.pyrDown(pyramids_old[-1]))
        pyramids_new.append(cv2.pyrDown(pyramids_new[-1]))

    # 各ピラミッドレベルでオプティカルフローを計算
    for level in reversed(range(levels)):
        level_flow = np.zeros_like(flow)
        pyramid_old = pyramids_old[level]
        pyramid_new = pyramids_new[level]

        scale = 2 ** level
        current_window_size = window_size // scale

        for iteration in range(num_iterations):
            for y in range(0, pyramid_old.shape[0] - current_window_size, current_window_size):
                for x in range(0, pyramid_old.shape[1] - current_window_size, current_window_size):
                    # 窓を切り出して差分を取る
                    old_patch = pyramid_old[y:y + current_window_size, x:x + current_window_size]
                    new_patch = pyramid_new[y:y + current_window_size, x:x + current_window_size]

                    # 勾配計算
                    grad_x = cv2.Sobel(old_patch, cv2.CV_32F, 1, 0, ksize=poly_n)
                    grad_y = cv2.Sobel(old_patch, cv2.CV_32F, 0, 1, ksize=poly_n)
                    grad_t = (new_patch.astype(np.float32) - old_patch.astype(np.float32))

                    # 最小二乗法で移動ベクトルを計算
                    A = np.stack((grad_x.ravel(), grad_y.ravel()), axis=1)
                    b = -grad_t.ravel()

                    # 方程式を解く
                    if A.shape[0] > 2:  # 方程式の解を得るためには少なくとも2つの式が必要
                        nu = np.linalg.lstsq(A, b, rcond=None)[0]
                        level_flow[y:y + current_window_size, x:x + current_window_size, 0] += nu[0]
                        level_flow[y:y + current_window_size, x:x + current_window_size, 1] += nu[1]

        # 高い解像度のフローに追加
        if level != levels - 1:
            level_flow = cv2.resize(level_flow, (flow.shape[1], flow.shape[0]))
        flow += level_flow

    # ガウシアンフィルタで平滑化
    flow[:, :, 0] = cv2.GaussianBlur(flow[:, :, 0], (poly_n, poly_n), poly_sigma)
    flow[:, :, 1] = cv2.GaussianBlur(flow[:, :, 1], (poly_n, poly_n), poly_sigma)

    return flow


while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 現在のフレームをグレースケールに変換
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # フルスクラッチでFarneback法に基づくオプティカルフローを計算
    flow = calc_optical_flow_farneback_scratch(old_gray, frame_gray, num_iterations=5, window_size=21, poly_n=7, poly_sigma=1.5)

    # フローのベクトルを描画
    h, w = frame_gray.shape[:2]
    step = 8  # より小さなステップでサンプリングして精度を向上させる
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx * 100, y + fy * 100]).T.reshape(-1, 2, 2)
    lines = np.int32(lines)
    for (x1, y1), (x2, y2) in lines:
        # mask = cv2.line(mask, (x1, y1), (x2, y2), color[np.random.randint(0, len(color))].tolist(), 1)
        # frame = cv2.circle(frame, (x1, y1), 1, color[np.random.randint(0, len(color))].tolist(), -1)
        mask = cv2.line(mask, (x1, y1), (x2, y2), (128, 128, 128), 1)
        frame = cv2.circle(frame, (x1, y1), 1, (128, 128, 128), -1)

    img = cv2.add(frame, mask)

    # 結果を表示
    print('Farneback Optical Flow - High Precision (Scratch)')
    cv2_imshow(img)


    # 次のフレームの準備
    old_gray = frame_gray.copy()

# リリースとウィンドウの破棄
cap.release()

## オプティカルフローの標準実装

### *等速円運動*

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 動画のキャプチャを開始
cap = cv2.VideoCapture('uniformMotionWithNoEdit.mp4')

# ランダムな色を生成
color = np.random.randint(0, 255, (100, 3))

# 最初のフレームを読み込み、グレースケールに変換
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# 描画用のマスクを作成
mask = np.zeros_like(old_frame)

def evaluate_optical_flow(flow, true_velocity, step=8):
    # フローをサンプリングし、理論的な速度と比較して誤差を計算する
    h, w = flow.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # 理論的な速度ベクトルとの誤差を計算
    true_fx, true_fy = true_velocity
    error_x = fx - true_fx
    error_y = fy - true_fy
    mse = np.mean(error_x**2 + error_y**2)
    return mse

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 現在のフレームをグレースケールに変換
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # OpenCVのFarneback法を使用してオプティカルフローを計算
    flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray, None, 0.5, 3, 50, 3, 5, 1.2, 256)

    # フローのベクトルを描画
    h, w = frame_gray.shape[:2]
    step = 8  # より小さなステップでサンプリングして精度を向上させる
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx * 10, y + fy * 10]).T.reshape(-1, 2, 2)
    lines = np.int32(lines)
    for (x1, y1), (x2, y2) in lines:
        # mask = cv2.line(mask, (x1, y1), (x2, y2), color[np.random.randint(0, len(color))].tolist(), 1)
        # frame = cv2.circle(frame, (x1, y1), 1, color[np.random.randint(0, len(color))].tolist(), -1)
        mask = cv2.line(mask, (x1, y1), (x2, y2), (128, 128, 128), 1)
        frame = cv2.circle(frame, (x1, y1), 1, (128, 128, 128), -1)

    img = cv2.add(frame, mask)

    # 結果を表示
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 理論値との誤差を評価
    true_velocity = (1.0, 1.0)  # 等速度円運動の理論的な速度ベクトル（仮定）
    mse = evaluate_optical_flow(flow, true_velocity, step=8)
    print(f"Mean Squared Error (MSE) of Optical Flow: {mse}")

    # # 'q'キーで終了（Matplotlibは待機処理がないためループ継続確認を手動に）
    # user_input = input("Press 'q' to quit or any other key to continue: ")
    # if user_input.lower() == 'q':
    #     break

    # 隣接するフレーム間のオプティカルフローを求めるために、現在のフレームを次のフレームの基準として更新
    old_gray = frame_gray.copy()

# リリースとウィンドウの破棄
cap.release()


#### 円運動（フローを拡大）

In [ ]:
cap.release()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 動画のキャプチャを開始
cap = cv2.VideoCapture('uniformMotionWithNoEdit.mp4')

# ランダムな色を生成
color = np.random.randint(0, 255, (100, 3))

# 最初のフレームを読み込み、グレースケールに変換
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# 描画用のマスクを作成
mask = np.zeros_like(old_frame)

def evaluate_optical_flow(flow, true_velocity, step=8):
    # フローをサンプリングし、理論的な速度と比較して誤差を計算する
    h, w = flow.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # 理論的な速度ベクトルとの誤差を計算
    true_fx, true_fy = true_velocity
    error_x = fx - true_fx
    error_y = fy - true_fy
    mse = np.mean(error_x**2 + error_y**2)
    return mse

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 現在のフレームをグレースケールに変換
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # OpenCVのFarneback法を使用してオプティカルフローを計算
    flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # フローのベクトルを描画
    h, w = frame_gray.shape[:2]
    step = 8  # より小さなステップでサンプリングして精度を向上させる
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx * 100, y + fy * 100]).T.reshape(-1, 2, 2)
    lines = np.int32(lines)
    for (x1, y1), (x2, y2) in lines:
        # mask = cv2.line(mask, (x1, y1), (x2, y2), color[np.random.randint(0, len(color))].tolist(), 1)
        # frame = cv2.circle(frame, (x1, y1), 1, color[np.random.randint(0, len(color))].tolist(), -1)
        mask = cv2.line(mask, (x1, y1), (x2, y2), (128, 128, 128), 1)
        frame = cv2.circle(frame, (x1, y1), 1, (128, 128, 128), -1)

    img = cv2.add(frame, mask)

    # 結果を表示
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 理論値との誤差を評価
    true_velocity = (1.0, 1.0)  # 等速度円運動の理論的な速度ベクトル（仮定）
    mse = evaluate_optical_flow(flow, true_velocity, step=8)
    print(f"Mean Squared Error (MSE) of Optical Flow: {mse}")

    # # 'q'キーで終了（Matplotlibは待機処理がないためループ継続確認を手動に）
    # user_input = input("Press 'q' to quit or any other key to continue: ")
    # if user_input.lower() == 'q':
    #     break

    # 隣接するフレーム間のオプティカルフローを求めるために、現在のフレームを次のフレームの基準として更新
    old_gray = frame_gray.copy()

# リリースとウィンドウの破棄
cap.release()


## 座標変換のためのintに直さない場合

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 動画のキャプチャを開始
cap = cv2.VideoCapture('uniformMotionWithNoEdit.mp4')

# ランダムな色を生成
color = np.random.randint(0, 255, (100, 3))

# 最初のフレームを読み込み、グレースケールに変換
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# 描画用のマスクを作成
mask = np.zeros_like(old_frame)

def evaluate_optical_flow(flow, true_velocity, step=8):
    # フローをサンプリングし、理論的な速度と比較して誤差を計算する
    h, w = flow.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # 理論的な速度ベクトルとの誤差を計算
    true_fx, true_fy = true_velocity
    error_x = fx - true_fx
    error_y = fy - true_fy
    mse = np.mean(error_x**2 + error_y**2)
    return mse

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 現在のフレームをグレースケールに変換
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # OpenCVのFarneback法を使用してオプティカルフローを計算
    flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # フローのベクトルを描画
    h, w = frame_gray.shape[:2]
    step = 8  # より小さなステップでサンプリングして精度を向上させる
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx * 100, y + fy * 100]).T.reshape(-1, 2, 2)
    print(lines)
    lines = np.int64(lines)
    print(lines)
    for (x1, y1), (x2, y2) in lines:
        # mask = cv2.line(mask, (x1, y1), (x2, y2), color[np.random.randint(0, len(color))].tolist(), 1)
        # frame = cv2.circle(frame, (x1, y1), 1, color[np.random.randint(0, len(color))].tolist(), -1)
        mask = cv2.line(mask, (x1, y1), (x2, y2), (128, 128, 128), 1)
        frame = cv2.circle(frame, (x1, y1), 1, (128, 128, 128), -1)

    img = cv2.add(frame, mask)

    # 結果を表示
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 理論値との誤差を評価
    true_velocity = (1.0, 1.0)  # 等速度円運動の理論的な速度ベクトル（仮定）
    mse = evaluate_optical_flow(flow, true_velocity, step=8)
    print(f"Mean Squared Error (MSE) of Optical Flow: {mse}")

    # # 'q'キーで終了（Matplotlibは待機処理がないためループ継続確認を手動に）
    # user_input = input("Press 'q' to quit or any other key to continue: ")
    # if user_input.lower() == 'q':
    #     break

    # 隣接するフレーム間のオプティカルフローを求めるために、現在のフレームを次のフレームの基準として更新
    old_gray = frame_gray.copy()

# リリースとウィンドウの破棄
cap.release()


### 垂直落下運動

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 動画のキャプチャを開始
cap = cv2.VideoCapture('fall_ball.mp4')

# ランダムな色を生成
color = np.random.randint(0, 255, (100, 3))

# 最初のフレームを読み込み、グレースケールに変換
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# 描画用のマスクを作成
mask = np.zeros_like(old_frame)

def evaluate_optical_flow(flow, true_velocity, step=8):
    # フローをサンプリングし、理論的な速度と比較して誤差を計算する
    h, w = flow.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # 理論的な速度ベクトルとの誤差を計算
    true_fx, true_fy = true_velocity
    error_x = fx - true_fx
    error_y = fy - true_fy
    mse = np.mean(error_x**2 + error_y**2)
    return mse

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 現在のフレームをグレースケールに変換
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # OpenCVのFarneback法を使用してオプティカルフローを計算
    flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray, None, 0.5, 3, 50, 3, 5, 1.2, 256)

    # フローのベクトルを描画
    h, w = frame_gray.shape[:2]
    step = 8  # より小さなステップでサンプリングして精度を向上させる
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx * 100, y + fy * 100]).T.reshape(-1, 2, 2)
    lines = np.int32(lines)
    for (x1, y1), (x2, y2) in lines:
        # mask = cv2.line(mask, (x1, y1), (x2, y2), color[np.random.randint(0, len(color))].tolist(), 1)
        # frame = cv2.circle(frame, (x1, y1), 1, color[np.random.randint(0, len(color))].tolist(), -1)
        mask = cv2.line(mask, (x1, y1), (x2, y2), (128, 128, 128), 1)
        frame = cv2.circle(frame, (x1, y1), 1, (128, 128, 128), -1)

    img = cv2.add(frame, mask)

    # 結果を表示
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 理論値との誤差を評価
    true_velocity = (1.0, 1.0)  # 等速度円運動の理論的な速度ベクトル（仮定）
    mse = evaluate_optical_flow(flow, true_velocity, step=8)
    print(f"Mean Squared Error (MSE) of Optical Flow: {mse}")

    # # 'q'キーで終了（Matplotlibは待機処理がないためループ継続確認を手動に）
    # user_input = input("Press 'q' to quit or any other key to continue: ")
    # if user_input.lower() == 'q':
    #     break

    # 隣接するフレーム間のオプティカルフローを求めるために、現在のフレームを次のフレームの基準として更新
    old_gray = frame_gray.copy()

# リリースとウィンドウの破棄
cap.release()


#### 重心のみ(この場合ソースコードがおかしい可能性あり。標準よりも精度が低い)重心情報飲み使えているかチェック

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

# 動画のキャプチャを開始
cap = cv2.VideoCapture('fall_ball.mp4')

# ランダムな色を生成
color = np.random.randint(0, 255, (100, 3))

# 最初のフレームを読み込み、グレースケールに変換
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# 描画用のマスクを作成
mask = np.zeros_like(old_frame)

def evaluate_optical_flow(flow, true_velocity_x, true_velocity_y, step=8):
    # フローをサンプリングし、理論的な速度と比較して誤差を計算する
    h, w = flow.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # 理論的な速度ベクトルとの誤差を角度の違いで計算
    true_fx = true_velocity_x[y, x]
    true_fy = true_velocity_y[y, x]
    dot_product = fx * true_fx + fy * true_fy
    magnitude_flow = np.sqrt(fx**2 + fy**2)
    magnitude_true = np.sqrt(true_fx**2 + true_fy**2)
    cos_theta = dot_product / (magnitude_flow * magnitude_true + 1e-5)  # 小さな値を足してゼロ除算を防止
    angle_error = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # コサイン値をクリップして範囲を保つ
    mean_angle_error = np.degrees(np.mean(angle_error))  # 角度誤差の平均を度数法で返す
    return mean_angle_error

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 現在のフレームをグレースケールに変換
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 物体の重心を計算
    ret, thresh = cv2.threshold(frame_gray, 127, 255, cv2.THRESH_BINARY)
    M = cv2.moments(thresh)
    if M['m00'] != 0:
        centroid_x = int(M['m10'] / M['m00'])
        centroid_y = int(M['m01'] / M['m00'])
    else:
        centroid_x, centroid_y = 0, 0

    # 重心以外の部分をマスク処理して物体を検知できなくする
    mask_centroid = np.ones_like(frame_gray) * 255
    cv2.circle(mask_centroid, (centroid_x, centroid_y), 20, 0, -1)  # 重心の周りを除いてマスク
    frame_gray_masked = cv2.bitwise_and(frame_gray, frame_gray, mask=mask_centroid)

    # OpenCVのFarneback法を使用してオプティカルフローを計算
    flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray_masked, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # フローのベクトルを描画
    h, w = frame_gray.shape[:2]
    step = 8  # より小さなステップでサンプリングして精度を向上させる
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx * 10, y + fy * 10]).T.reshape(-1, 2, 2)
    lines = np.int32(lines)
    for (x1, y1), (x2, y2) in lines:
        # mask = cv2.line(mask, (x1, y1), (x2, y2), color[np.random.randint(0, len(color))].tolist(), 1)
        # frame = cv2.circle(frame, (x1, y1), 1, color[np.random.randint(0, len(color))].tolist(), -1)
        mask = cv2.line(mask, (x1, y1), (x2, y2), (128, 128, 128), 1)
        frame = cv2.circle(frame, (x1, y1), 1, (128, 128, 128), -1)
    img = cv2.add(frame, mask)

    # 結果を表示
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 隣接するフレーム間のオプティカルフローを求めるために、現在のフレームを次のフレームの基準として更新
    old_gray = frame_gray.copy()

    # # 'q'キーで終了（Matplotlibは待機処理がないためループ継続確認を手動に）
    # user_input = input("Press 'q' to quit or any other key to continue: ")
    # if user_input.lower() == 'q':
    #     break

# リリースとウィンドウの破棄
cap.release()

#### ベクトルの大きさ通常

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 動画のキャプチャを開始
cap = cv2.VideoCapture('fall_ball.mp4')

# ランダムな色を生成
color = np.random.randint(0, 255, (100, 3))

# 最初のフレームを読み込み、グレースケールに変換
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# 描画用のマスクを作成
mask = np.zeros_like(old_frame)

def evaluate_optical_flow(flow, true_velocity, step=8):
    # フローをサンプリングし、理論的な速度と比較して誤差を計算する
    h, w = flow.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # 理論的な速度ベクトルとの誤差を計算
    true_fx, true_fy = true_velocity
    error_x = fx - true_fx
    error_y = fy - true_fy
    mse = np.mean(error_x**2 + error_y**2)
    return mse

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 現在のフレームをグレースケールに変換
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # OpenCVのFarneback法を使用してオプティカルフローを計算
    flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray, None, 0.5, 3, 50, 3, 5, 1.2, 256)

    # フローのベクトルを描画
    h, w = frame_gray.shape[:2]
    step = 8  # より小さなステップでサンプリングして精度を向上させる
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx * 10, y + fy * 10]).T.reshape(-1, 2, 2)
    lines = np.int32(lines)
    for (x1, y1), (x2, y2) in lines:
        # mask = cv2.line(mask, (x1, y1), (x2, y2), color[np.random.randint(0, len(color))].tolist(), 1)
        # frame = cv2.circle(frame, (x1, y1), 1, color[np.random.randint(0, len(color))].tolist(), -1)
        mask = cv2.line(mask, (x1, y1), (x2, y2), (128, 128, 128), 1)
        frame = cv2.circle(frame, (x1, y1), 1, (128, 128, 128), -1)

    img = cv2.add(frame, mask)

    # 結果を表示
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 理論値との誤差を評価
    true_velocity = (1.0, 1.0)  # 等速度円運動の理論的な速度ベクトル（仮定）
    mse = evaluate_optical_flow(flow, true_velocity, step=8)
    print(f"Mean Squared Error (MSE) of Optical Flow: {mse}")

    # # 'q'キーで終了（Matplotlibは待機処理がないためループ継続確認を手動に）
    # user_input = input("Press 'q' to quit or any other key to continue: ")
    # if user_input.lower() == 'q':
    #     break

    # 隣接するフレーム間のオプティカルフローを求めるために、現在のフレームを次のフレームの基準として更新
    old_gray = frame_gray.copy()

# リリースとウィンドウの破棄
cap.release()


### 理論値と標準実装の比較

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

# 動画のキャプチャを開始
cap = cv2.VideoCapture('uniformMotionWithNoEdit.mp4')

# ランダムな色を生成
color = np.random.randint(0, 255, (100, 3))

# 最初のフレームを読み込み、グレースケールに変換
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# 描画用のマスクを作成
mask = np.zeros_like(old_frame)

def evaluate_optical_flow(flow, true_velocity, step=8):
    # フローをサンプリングし、理論的な速度と比較して誤差を計算する
    h, w = flow.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # 理論的な速度ベクトルとの誤差を計算
    true_fx, true_fy = true_velocity
    error_x = fx - true_fx
    error_y = fy - true_fy
    mse = np.mean(error_x**2 + error_y**2)
    return mse

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 現在のフレームをグレースケールに変換
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # OpenCVのFarneback法を使用してオプティカルフローを計算
    flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # フローのベクトルを描画
    h, w = frame_gray.shape[:2]
    step = 8  # より小さなステップでサンプリングして精度を向上させる
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx * 10, y + fy * 10]).T.reshape(-1, 2, 2)
    lines = np.int32(lines)
    for (x1, y1), (x2, y2) in lines:
        mask = cv2.line(mask, (x1, y1), (x2, y2), color[np.random.randint(0, len(color))].tolist(), 1)
        frame = cv2.circle(frame, (x1, y1), 1, color[np.random.randint(0, len(color))].tolist(), -1)

    img = cv2.add(frame, mask)

    # 理論的な円運動の速度ベクトルを計算し描画
    radius = 200  # 仮定された円運動の半径
    omega = 1/200 # 角速度（ラジアン毎フレーム）
    center_x, center_y = w // 2, h // 2  # 円運動の中心をフレームの中心と仮定
    true_fx = -radius * omega * np.sin(omega * np.arange(len(x)))
    true_fy = radius * omega * np.cos(omega * np.arange(len(y)))
    for i in range(len(x)):
        true_x, true_y = x[i], y[i]
        img = cv2.arrowedLine(img, (true_x, true_y), (int(true_x + true_fx[i] * 10), int(true_y + true_fy[i] * 10)), (0, 255, 0), 1, tipLength=0.3)

    # 結果を表示
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 理論値との誤差を評価
    mse = evaluate_optical_flow(flow, (true_fx.mean(), true_fy.mean()), step=8)
    print(f"Mean Squared Error (MSE) of Optical Flow: {mse}")

    # # 'q'キーで終了（Matplotlibは待機処理がないためループ継続確認を手動に）
    # user_input = input("Press 'q' to quit or any other key to continue: ")
    # if user_input.lower() == 'q':
    #     break

    # 隣接するフレーム間のオプティカルフローを求めるために、現在のフレームを次のフレームの基準として更新
    old_gray = frame_gray.copy()

# リリースとウィンドウの破棄
cap.release()


### 理論値とベクトルの向きで比較

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

# 動画のキャプチャを開始
cap = cv2.VideoCapture('uniformMotionWithNoEdit.mp4')

# ランダムな色を生成
color = np.random.randint(0, 255, (100, 3))

# 最初のフレームを読み込み、グレースケールに変換
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# 描画用のマスクを作成
mask = np.zeros_like(old_frame)

def evaluate_optical_flow(flow, true_velocity, step=8):
    # フローをサンプリングし、理論的な速度と比較して誤差を計算する
    h, w = flow.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # 理論的な速度ベクトルとの誤差を角度の違いで計算
    true_fx, true_fy = true_velocity
    dot_product = fx * true_fx + fy * true_fy
    magnitude_flow = np.sqrt(fx**2 + fy**2)
    magnitude_true = np.sqrt(true_fx**2 + true_fy**2)
    cos_theta = dot_product / (magnitude_flow * magnitude_true + 1e-5)  # 小さな値を足してゼロ除算を防止
    angle_error = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # コサイン値をクリップして範囲を保つ
    mean_angle_error = np.degrees(np.mean(angle_error))  # 角度誤差の平均を度数法で返す
    return mean_angle_error

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 現在のフレームをグレースケールに変換
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # OpenCVのFarneback法を使用してオプティカルフローを計算
    flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # フローのベクトルを描画
    h, w = frame_gray.shape[:2]
    step = 8  # より小さなステップでサンプリングして精度を向上させる
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx * 10, y + fy * 10]).T.reshape(-1, 2, 2)
    lines = np.int32(lines)
    for (x1, y1), (x2, y2) in lines:
        mask = cv2.line(mask, (x1, y1), (x2, y2), color[np.random.randint(0, len(color))].tolist(), 1)
        frame = cv2.circle(frame, (x1, y1), 1, color[np.random.randint(0, len(color))].tolist(), -1)

    img = cv2.add(frame, mask)

    # 理論的な円運動の速度ベクトルを計算し描画
    radius = 200  # 仮定された円運動の半径
    omega = 1/200  # 角速度（ラジアン毎フレーム）
    center_x, center_y = w // 2, h // 2  # 円運動の中心をフレームの中心と仮定
    true_fx = -radius * omega * np.sin(omega * np.arange(len(x)))
    true_fy = radius * omega * np.cos(omega * np.arange(len(y)))
    for i in range(len(x)):
        true_x, true_y = x[i], y[i]
        img = cv2.arrowedLine(img, (true_x, true_y), (int(true_x + true_fx[i] * 10), int(true_y + true_fy[i] * 10)), (0, 255, 0), 1, tipLength=0.3)

    # 結果を表示
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 理論値との誤差を評価（ベクトルの向きの誤差）
    mean_angle_error = evaluate_optical_flow(flow, (true_fx.mean(), true_fy.mean()), step=8)
    print(f"Mean Angular Error (in degrees) of Optical Flow: {mean_angle_error}")


    # 隣接するフレーム間のオプティカルフローを求めるために、現在のフレームを次のフレームの基準として更新
    old_gray = frame_gray.copy()

# リリースとウィンドウの破棄
cap.release()


### 評価指標修正版

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

# 動画のキャプチャを開始
cap = cv2.VideoCapture('input_video.mp4')

# ランダムな色を生成
color = np.random.randint(0, 255, (100, 3))

# 最初のフレームを読み込み、グレースケールに変換
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# 描画用のマスクを作成
mask = np.zeros_like(old_frame)

def evaluate_optical_flow(flow, true_velocity_x, true_velocity_y, step=8):
    # フローをサンプリングし、理論的な速度と比較して誤差を計算する
    h, w = flow.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # 理論的な速度ベクトルとの誤差を角度の違いで計算
    true_fx = true_velocity_x[y, x]
    true_fy = true_velocity_y[y, x]
    dot_product = fx * true_fx + fy * true_fy
    magnitude_flow = np.sqrt(fx**2 + fy**2)
    magnitude_true = np.sqrt(true_fx**2 + true_fy**2)
    cos_theta = dot_product / (magnitude_flow * magnitude_true + 1e-5)  # 小さな値を足してゼロ除算を防止
    angle_error = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # コサイン値をクリップして範囲を保つ
    mean_angle_error = np.degrees(np.mean(angle_error))  # 角度誤差の平均を度数法で返す
    return mean_angle_error

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 現在のフレームをグレースケールに変換
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # OpenCVのFarneback法を使用してオプティカルフローを計算
    flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # フローのベクトルを描画
    h, w = frame_gray.shape[:2]
    step = 8  # より小さなステップでサンプリングして精度を向上させる
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx * 10, y + fy * 10]).T.reshape(-1, 2, 2)
    lines = np.int32(lines)
    for (x1, y1), (x2, y2) in lines:
        mask = cv2.line(mask, (x1, y1), (x2, y2), color[np.random.randint(0, len(color))].tolist(), 1)
        frame = cv2.circle(frame, (x1, y1), 1, color[np.random.randint(0, len(color))].tolist(), -1)

    img = cv2.add(frame, mask)

    # 理論的な円運動の速度ベクトルを計算
    radius = 200  # 仮定された円運動の半径
    omega = 1/200  # 角速度（ラジアン毎フレーム）
    center_x, center_y = w // 2, h // 2  # 円運動の中心をフレームの中心と仮定
    true_velocity_x = -omega * (y - center_y)  # 理論的な速度ベクトルのX成分
    true_velocity_y = omega * (x - center_x)   # 理論的な速度ベクトルのY成分

    # 理論的な円運動の速度ベクトルを描画
    for i in range(len(x)):
        true_x, true_y = x[i], y[i]
        img = cv2.arrowedLine(img, (true_x, true_y), (int(true_x + true_velocity_x[i] * 10), int(true_y + true_velocity_y[i] * 10)), (0, 255, 0), 1, tipLength=0.3)

    # 結果を表示
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 理論値との誤差を評価（ベクトルの向きの誤差）
    mean_angle_error = evaluate_optical_flow(flow, true_velocity_x, true_velocity_y, step=8)
    print(f"Mean Angular Error (in degrees) of Optical Flow: {mean_angle_error}")

    # # 'q'キーで終了（Matplotlibは待機処理がないためループ継続確認を手動に）
    # user_input = input("Press 'q' to quit or any other key to continue: ")
    # if user_input.lower() == 'q':
    #     break

    # 隣接するフレーム間のオプティカルフローを求めるために、現在のフレームを次のフレームの基準として更新
    old_gray = frame_gray.copy()

# リリースとウィンドウの破棄
cap.release()

### Farneback 改良版

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

# 動画のキャプチャを開始
cap = cv2.VideoCapture('uniformMotionWithNoEdit.mp4')

# ランダムな色を生成
color = np.random.randint(0, 255, (100, 3))

# 最初のフレームを読み込み、グレースケールに変換
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# 描画用のマスクを作成
mask = np.zeros_like(old_frame)
# cv2_imshow(mask)

def evaluate_optical_flow(flow, true_velocity_x, true_velocity_y, step=8):
    # フローをサンプリングし、理論的な速度と比較して誤差を計算する
    h, w = flow.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # 理論的な速度ベクトルとの誤差を角度の違いで計算
    true_fx = true_velocity_x[y, x]
    true_fy = true_velocity_y[y, x]
    dot_product = fx * true_fx + fy * true_fy
    magnitude_flow = np.sqrt(fx**2 + fy**2)
    magnitude_true = np.sqrt(true_fx**2 + true_fy**2)
    cos_theta = dot_product / (magnitude_flow * magnitude_true + 1e-5)  # 小さな値を足してゼロ除算を防止
    angle_error = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # コサイン値をクリップして範囲を保つ
    mean_angle_error = np.degrees(np.mean(angle_error))  # 角度誤差の平均を度数法で返す
    return mean_angle_error

def calc_optical_flow_farneback(prev, next, pyr_scale=0.5, levels=3, winsize=15, iterations=3, poly_n=5, poly_sigma=1.1, flags=0):
    # フローを格納するための初期化
    flow = np.zeros((prev.shape[0], prev.shape[1], 2), dtype=np.float32)

    # ピラミッドを作成
    pyramid_prev = [prev]
    pyramid_next = [next]
    for level in range(1, levels):
        pyramid_prev.append(cv2.pyrDown(pyramid_prev[-1]))
        pyramid_next.append(cv2.pyrDown(pyramid_next[-1]))

    # 最も小さいレベルから順に処理
    for level in reversed(range(levels)):
        # 現在のレベルの画像
        prev_level = pyramid_prev[level]
        next_level = pyramid_next[level]

        # 拡大処理（最初のレベル以外）
        if level < levels - 1:
            flow = cv2.pyrUp(flow)
            flow = cv2.resize(flow, (prev_level.shape[1], prev_level.shape[0]))

        # 各ピクセルの動きを計算
        h, w = prev_level.shape[:2]
        A_sum = np.zeros((2, 2), dtype=np.float32)
        b_sum = np.zeros(2, dtype=np.float32)
        w_sum = 0
        for y in range(h):
            for x in range(w):
                # 値がオーバーフローしないように型変換
                prev_pixel = np.float32(prev_level[y, x])
                next_pixel = np.float32(next_level[y, x])
                Ix = (np.float32(next_level[y, min(x + 1, w - 1)]) - np.float32(next_level[y, max(x - 1, 0)])) / 2.0
                Iy = (np.float32(next_level[min(y + 1, h - 1), x]) - np.float32(next_level[max(y - 1, 0), x])) / 2.0
                It = next_pixel - prev_pixel
                A = np.array([[Ix * Ix, Ix * Iy], [Ix * Iy, Iy * Iy]], dtype=np.float32)
                b = np.array([-Ix * It, -Iy * It], dtype=np.float32)
                weight = np.exp(-(x**2 + y**2) / (2 * winsize**2))  # 重み関数（ガウシアン）
                A_sum += weight * A
                b_sum += weight * b
                w_sum += weight

        # 最小二乗法による解を求める
        if w_sum > 0:
            flow_vector = np.linalg.solve(A_sum + 1e-5 * np.eye(2), b_sum)  # 小さな値を足して安定化
            flow[:, :, 0] = flow_vector[0]
            flow[:, :, 1] = flow_vector[1]

    return flow

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 現在のフレームをグレースケールに変換
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # cv2_imshow(frame_gray)
    # 物体の重心を計算
    ret, thresh = cv2.threshold(frame_gray, 127, 255, cv2.THRESH_BINARY)
    # cv2_imshow(thresh)
    M = cv2.moments(thresh)
    if M['m00'] != 0:
        centroid_x = int(M['m10'] / M['m00'])
        centroid_y = int(M['m01'] / M['m00'])
    else:
        centroid_x, centroid_y = 0, 0

    # 重心以外の部分をマスク処理して物体を検知できなくする
    mask_centroid = np.ones_like(frame_gray) * 0
    # cv2_imshow(mask_centroid)
    cv2.circle(mask_centroid, (centroid_x, centroid_y), 2, (128, 128, 128), -1)  # 重心の周りを除いてマスク
    frame_gray_masked = cv2.bitwise_and(frame_gray, frame_gray, mask=mask_centroid)
    # cv2_imshow(frame_gray_masked)

    # スクラッチ実装したFarneback法を使用してオプティカルフローを計算
    flow = calc_optical_flow_farneback(old_gray, frame_gray_masked, pyr_scale=0.5, levels=3, winsize=15, iterations=3, poly_n=5, poly_sigma=1.1, flags=0)

    # フローのベクトルを描画
    h, w = frame_gray.shape[:2]
    step = 8  # より小さなステップでサンプリングして精度を向上させる
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx * 1000, y + fy * 1000]).T.reshape(-1, 2, 2)
    lines = np.int32(lines)
    for (x1, y1), (x2, y2) in lines:
        # mask = cv2.line(mask, (x1, y1), (x2, y2), color[np.random.randint(0, len(color))].tolist(), 1)
        # frame = cv2.circle(frame, (x1, y1), 1, color[np.random.randint(0, len(color))].tolist(), -1)
        # mask = cv2.line(mask, (x1, y1), (x2, y2), (128, 128, 128), 1)
        # cv2_imshow(mask)
        frame = cv2.circle(frame, (x1, y1), 1, (128, 128, 128), -1)

    img = cv2.add(frame, mask)
    plt.imshow(img)
    # 結果を表示
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 隣接するフレーム間のオプティカルフローを求めるために、現在のフレームを次のフレームの基準として更新
    old_gray = frame_gray_masked.copy()

    # # 'q'キーで終了（Matplotlibは待機処理がないためループ継続確認を手動に）
    # user_input = input("Press 'q' to quit or any other key to continue: ")
    # if user_input.lower() == 'q':
    #     break

# リリースとウィンドウの破棄
cap.release()

### グレースケール変換のための配列タイプをfloat128にしたらエラーで怒られた?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, convolve

# Load the images
pathL = "./uniformMotionWithNoEdit/0001.png"
pathR = "./uniformMotionWithNoEdit/0002.png"

frameL = plt.imread(pathL)
frameR = plt.imread(pathR)

if frameL is None or frameR is None:
    print("Could not open or find one of the images.")
else:
    # Convert images to grayscale if needed and to float128 for higher precision
    if frameL.ndim == 3:
        frameL = np.dot(frameL[..., :3], [0.2989, 0.5870, 0.1140])
    if frameR.ndim == 3:
        frameR = np.dot(frameR[..., :3], [0.2989, 0.5870, 0.1140])

    frameL = frameL.astype(np.uint16)
    frameR = frameR.astype(np.float128)

    # Define parameters for optical flow calculation
    pyr_scale = 0.5
    levels = 3
    win_size = 15
    iterations = 3
    poly_n = 5
    poly_sigma = 1.2

    # Implement Farneback's optical flow method using NumPy
    def calc_optical_flow_farneback(prev, next, pyr_scale, levels, win_size, iterations, poly_n, poly_sigma):
        # Gaussian smoothing
        prev_blur = gaussian_filter(prev, sigma=poly_sigma)
        next_blur = gaussian_filter(next, sigma=poly_sigma)

        # Compute image gradients using Sobel filters
        sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float128)
        sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float128)

        Ix = convolve(prev_blur, sobel_x)
        Iy = convolve(prev_blur, sobel_y)
        It = next_blur - prev_blur

        # Initialize flow vectors
        u = np.zeros_like(prev, dtype=np.float128)
        v = np.zeros_like(prev, dtype=np.float128)

        # Iteratively update flow estimates
        for _ in range(iterations):
            u_avg = gaussian_filter(u, sigma=win_size / 6.0)
            v_avg = gaussian_filter(v, sigma=win_size / 6.0)

            # Calculate optical flow increment
            num = (Ix * u_avg + Iy * v_avg + It)
            denom = (Ix ** 2 + Iy ** 2 + 1e-10)

            delta_u = -Ix * num / denom
            delta_v = -Iy * num / denom

            u += delta_u
            v += delta_v

        return u, v

    # Calculate optical flow using NumPy implementation
    flowx, flowy = calc_optical_flow_farneback(frameL, frameR, pyr_scale, levels, win_size, iterations, poly_n, poly_sigma)

    # Colorize flow
    def map_val(x, a, b, c, d):
        x = max(min(x, b), a)
        return c + (d - c) * (x - a) / (b - a)

    def colorize_flow(u, v):
        u_min, u_max = np.min(u), np.max(u)
        v_min, v_max = np.min(v), np.max(v)
        u_min, u_max = abs(u_min), abs(u_max)
        v_min, v_max = abs(v_min), abs(v_max)
        d_max = max(u_min, u_max, v_min, v_max)

        h, w = u.shape
        dst = np.zeros((h, w, 3), dtype=np.uint8)

        for y in range(h):
            for x in range(w):
                dst[y, x, 0] = 0
                dst[y, x, 1] = int(map_val(-v[y, x], -d_max, d_max, 0, 255))
                dst[y, x, 2] = int(map_val(u[y, x], -d_max, d_max, 0, 255))

        return dst

    flow_image = colorize_flow(flowx, flowy)

    # Display the optical flow using matplotlib
    plt.imshow(flow_image)
    plt.title('Optical Flow')
    plt.axis('off')
    plt.show()

### 既存のオプティカルフローライブラリを使用すると、物体がどのように認識されているか？の確認。

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

# Load the images
pathL = "./uniformMotionWithNoEdit/0002.png"
pathR = "./uniformMotionWithNoEdit/0001.png"

frameL = cv2.imread(pathL, cv2.IMREAD_GRAYSCALE)
frameR = cv2.imread(pathR, cv2.IMREAD_GRAYSCALE)

if frameL is None or frameR is None:
    print("Could not open or find one of the images.")
else:
    # Calculate optical flow using Farneback's method with standard precision (32-bit float)
    flow = cv2.calcOpticalFlowFarneback(
        frameL.astype(np.float64), frameR.astype(np.float64), None, 0.5, 3, 15, 3, 5, 1.2, 0
    )

    # Split flow into horizontal and vertical components
    flowx, flowy = flow[..., 0], flow[..., 1]

    # Colorize flow
    def map_val(x, a, b, c, d):
        x = max(min(x, b), a)
        return c + (d - c) * (x - a) / (b - a)

    def colorize_flow(u, v):
        u_min, u_max = np.min(u), np.max(u)
        v_min, v_max = np.min(v), np.max(v)
        u_min, u_max = abs(u_min), abs(u_max)
        v_min, v_max = abs(v_min), abs(v_max)
        d_max = max(u_min, u_max, v_min, v_max)

        h, w = u.shape
        dst = np.zeros((h, w, 3), dtype=np.uint8)

        for y in range(h):
            for x in range(w):
                dst[y, x, 0] = 0
                dst[y, x, 1] = int(map_val(-v[y, x], -d_max, d_max, 0, 255))
                dst[y, x, 2] = int(map_val(u[y, x], -d_max, d_max, 0, 255))

        return dst

    flow_image = colorize_flow(flowx, flowy)

    print(flow)
    # Display the optical flow
    cv2_imshow(flow_image)
    # cv2.waitKey(0)
    cv2.destroyAllWindows()

##重心算出

### １フレームの重心のみ算出し表示する。

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from decimal import Decimal, getcontext

# Load the image using PIL
try:
    img = Image.open("uniformMotionWithNoEdit/0020.png")
except FileNotFoundError:
    print("Could not open or find the image")
    img = None

if img:
    # Convert the image to grayscale
    gray = img.convert("L")

    # Convert grayscale image to numpy array
    gray_np = np.array(gray)
    # Apply binary thresholding
    fg_mask = np.where(gray_np > 127, 255, 0)
    fg_mask = fg_mask.astype(np.uint8)
    # Find contours
    contours = []
    visited = np.zeros_like(fg_mask, dtype=bool)

    def dfs(x, y, contour):
        stack = [(x, y)]
        while stack:
            cx, cy = stack.pop()
            if visited[cx, cy]:
                continue
            visited[cx, cy] = True
            contour.append((cx, cy))
            for nx, ny in [(cx - 1, cy), (cx + 1, cy), (cx, cy - 1), (cx, cy + 1)]:
                if 0 <= nx < fg_mask.shape[0] and 0 <= ny < fg_mask.shape[1]:
                    if fg_mask[nx, ny] == 255 and not visited[nx, ny]:
                        stack.append((nx, ny))

    # Extract contours
    for i in range(fg_mask.shape[0]):
        for j in range(fg_mask.shape[1]):
            if fg_mask[i, j] == 255 and not visited[i, j]:
                contour = []
                dfs(i, j, contour)
                contours.append(contour)

    centroids = []
    print(contours)

    for contour in contours:
        print(contour)
        # Calculate centroid for each contour
        if contour:
            x_coords, y_coords = zip(*contour)
            centroid_x = sum(x_coords) / len(contour)
            centroid_y = sum(y_coords) / len(contour)
            print(type(centroid_x), type(centroid_y))
            centroids.append((centroid_x, centroid_y))

    # Display centroids
    print("Centroids:", centroids)
    # Optional visualization
    plt.imshow(gray_np, cmap='gray')
    for centroid in centroids:
        plt.plot(centroid[1], centroid[0], 'ro')
    plt.show()

# マスク処理

## 等速円運動の場合

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from google.colab.patches import cv2_imshow


cap = cv2.VideoCapture("uniformMotionWithNoEdit.mp4")
# bg_subtractor = cv2.createBackgroundSubtractorMOG2()
count = 0
while True:
  count += 1
  is_read, frame = cap.read()
  if not is_read:
    break
  # fg_mask = bg_subtractor.apply(frame)
  # _, fg_mask = cv2.threshold(fg_mask, 127, 255, cv2.THRESH_BINARY)
  gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  _, fg_mask = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_BINARY)
  # print(fg_mask.type)
  contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  centroids = []
  for cnt in contours:
    if cv2.contourArea(cnt) > 300:  # ignore small contours
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            centroids.append((cX, cY))
            cv2.circle(fg_mask, (cX, cY), 5, (0, 255, 0), -1)
  print(f"{count}フレーム目")
  cv2_imshow(fg_mask)


writer.release()
cap.release()


#### 落下運動の場合

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from google.colab.patches import cv2_imshow


cap = cv2.VideoCapture("fall_ball.mp4")
# bg_subtractor = cv2.createBackgroundSubtractorMOG2()
count = 0
while True:
  count += 1
  is_read, frame = cap.read()
  if not is_read:
    break
  # fg_mask = bg_subtractor.apply(frame)
  # _, fg_mask = cv2.threshold(fg_mask, 127, 255, cv2.THRESH_BINARY)
  gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  _, fg_mask = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_BINARY)
  # print(fg_mask.type)
  contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  centroids = []
  for cnt in contours:
    if cv2.contourArea(cnt) > 300:  # ignore small contours
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            centroids.append((cX, cY))
            cv2.circle(fg_mask, (cX, cY), 5, (0, 255, 0), -1)
  print(f"{count}フレーム目")
  cv2_imshow(fg_mask)


writer.release()
cap.release()

## 振り子運動

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from google.colab.patches import cv2_imshow


cap = cv2.VideoCapture("single_loop_ball.mp4")
# bg_subtractor = cv2.createBackgroundSubtractorMOG2()
count = 0
while True:
  count += 1
  is_read, frame = cap.read()
  if not is_read:
    break
  # fg_mask = bg_subtractor.apply(frame)
  # _, fg_mask = cv2.threshold(fg_mask, 127, 255, cv2.THRESH_BINARY)
  gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  _, fg_mask = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_BINARY)
  # print(fg_mask.type)
  contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  centroids = []
  for cnt in contours:
    if cv2.contourArea(cnt) > 300:  # ignore small contours
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            centroids.append((cX, cY))
            cv2.circle(fg_mask, (cX, cY), 5, (0, 255, 0), -1)
  print(f"{count}フレーム目")
  cv2_imshow(fg_mask)


writer.release()
cap.release()

## 床の摩擦を含む壁衝突運動

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from google.colab.patches import cv2_imshow


cap = cv2.VideoCapture("bound_ball_mov.mp4")
# bg_subtractor = cv2.createBackgroundSubtractorMOG2()
count = 0
while True:
  count += 1
  is_read, frame = cap.read()
  if not is_read:
    break
  # fg_mask = bg_subtractor.apply(frame)
  # _, fg_mask = cv2.threshold(fg_mask, 127, 255, cv2.THRESH_BINARY)
  gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  _, fg_mask = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_BINARY)
  # print(fg_mask.type)
  contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  centroids = []
  for cnt in contours:
    if cv2.contourArea(cnt) > 300:  # ignore small contours
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            centroids.append((cX, cY))
            cv2.circle(fg_mask, (cX, cY), 5, (0, 255, 0), -1)
  print(f"{count}フレーム目")
  cv2_imshow(fg_mask)


writer.release()
cap.release()

## ばね運動

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from google.colab.patches import cv2_imshow


cap = cv2.VideoCapture("bound_ball_with_damp.mp4")
# bg_subtractor = cv2.createBackgroundSubtractorMOG2()
count = 0
while True:
  count += 1
  is_read, frame = cap.read()
  if not is_read:
    break
  # fg_mask = bg_subtractor.apply(frame)
  # _, fg_mask = cv2.threshold(fg_mask, 127, 255, cv2.THRESH_BINARY)
  gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  _, fg_mask = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_BINARY)
  # print(fg_mask.type)
  contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  centroids = []
  for cnt in contours:
    if cv2.contourArea(cnt) > 300:  # ignore small contours
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            centroids.append((cX, cY))
            cv2.circle(fg_mask, (cX, cY), 5, (0, 255, 0), -1)
  print(f"{count}フレーム目")
  cv2_imshow(fg_mask)


writer.release()
cap.release()

# Franeback 従来

## 既存のオプティカルフローの手法

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from google.colab.patches import cv2_imshow


cap = cv2.VideoCapture("fall_ball.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

writer = cv2.VideoWriter("oldOutput_fall_ball_ver1.mp4", fourcc, fps, (width,height))
# bg_subtractor = cv2.createBackgroundSubtractorMOG2()

t1_frame = None
t2_frame = None

def sum_flow_elements(flow, y, h, x, w):
    region = flow[y:y+h, x:x+w]
    total = np.sum(region, axis=(0, 1))
    return total
count = 0
while True:
  count += 1
  is_read, frame = cap.read()
  if not is_read:
    break
  # fg_mask = bg_subtractor.apply(frame)
  # _, fg_mask = cv2.threshold(fg_mask, 127, 255, cv2.THRESH_BINARY)
  gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  _, fg_mask = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_BINARY)
  # print(f"{count}フレーム目")
  # cv2_imshow(fg_mask)
  # print(fg_mask.type)
  contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  centroids = []
  for cnt in contours:
    if cv2.contourArea(cnt) > 300:  # ignore small contours
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            centroids.append((cX, cY))
            # cv2.circle(frame, (cX, cY), 5, (255, 255, 255), -1)
  if (t1_frame is not None) and ( t2_frame is not None):
    frame_opt = t2_frame.copy()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    t1_frame_gray = cv2.cvtColor(t1_frame, cv2.COLOR_BGR2GRAY)
    t2_frame_gray = cv2.cvtColor(t2_frame, cv2.COLOR_BGR2GRAY)
    _, fg_mask = cv2.threshold(frame_gray, 127, 255, cv2.THRESH_BINARY)
    _, fg_mask1 = cv2.threshold(t1_frame_gray, 127, 255, cv2.THRESH_BINARY)
    _, fg_mask2 = cv2.threshold(t2_frame_gray, 127, 255, cv2.THRESH_BINARY)
    flow = cv2.calcOpticalFlowFarneback(fg_mask1, fg_mask2, None, 0.5, 3, 30, 3, 3, 1.1, 0)
    nextFlow = cv2.calcOpticalFlowFarneback(fg_mask2, fg_mask, None, 0.5, 3, 30, 3, 3, 1.1, 0)
    Flow = nextFlow - flow

    totalAOF = sum_flow_elements(Flow, 0, height, 0, width)
    for cX, cY in preCentroids:
        point_st = (cX, cY)
        point_ed = (cX + int(totalAOF[0]), cY + int(totalAOF[1]))
        cv2.line(frame_opt, point_st, point_ed, (255, 0, 0), 2)
    preCentroids = centroids.copy()
    writer.write(frame_opt)
    t1_frame = t2_frame.copy()
    t2_frame = frame.copy()
  elif t1_frame is None and t2_frame is not None:
        t1_frame = t2_frame.copy()
        t2_frame = frame.copy()
        writer.write(t2_frame)
        preCentroids = centroids.copy()
  elif t2_frame is None:
    t2_frame = frame.copy()
    writer.write(t2_frame)

writer.release()
cap.release()


### 重心計算方法

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

img = cv2.imread("uniformMotionWithNoEdit/0001.png")
if img is None:
  print("Could not open or find the image")
# cv2_imshow(img)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

_, fg_mask = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
centroids = []
print(contours)
for cnt in contours:
    if cv2.contourArea(cnt) > 500:  # ignore small contours
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            centroids.append((cX, cY))
            cv2.circle(img, (cX, cY), 5, (0, 255, 0), -1)


cv2_imshow(img)
print(f'母数{M["m00"]},横重心：{M["m10"]},縦重心：{M["m01"]}')

# 提案手法(提案：青木教授・実装：坂本)
- 重心のみの速度ベクトルと角度ベクトルを求める.

In [ ]:
import cv2
import numpy as np


cap = cv2.VideoCapture("uniformMotionWithNoEdit.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

writer = cv2.VideoWriter("output_uniformMotionWithNoEdit_ver3_gausian_filter.mp4", fourcc, fps, (width, height))

bg_subtractor = cv2.createBackgroundSubtractorMOG2()

def sum_flow_elements(flow, y, h, x, w):
    region = flow[y:y+h, x:x+w]
    total = np.sum(region, axis=(0, 1))
    return total

t1_frame = None
t2_frame = None

while True:
    is_read, frame = cap.read()
    if not is_read:
        break

    # fg_mask = bg_subtractor.apply(frame)
    # _, fg_mask = cv2.threshold(fg_mask, 127, 255, cv2.THRESH_BINARY)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, fg_mask = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    centroids = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 300:  # ignore small contours
            M = cv2.moments(cnt)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                centroids.append((cX, cY))
                cv2.circle(frame, (cX, cY), 5, (0, 0, 255), -1)
    if t1_frame is not None and t2_frame is not None:
        frame_opt = t2_frame.copy()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        t1_frame_gray = cv2.cvtColor(t1_frame, cv2.COLOR_BGR2GRAY)
        t2_frame_gray = cv2.cvtColor(t2_frame, cv2.COLOR_BGR2GRAY)
        # cv2_imshow(frame_gray)
        # _, fg_mask = cv2.threshold(frame_gray, 127, 255, cv2.THRESH_BINARY)
        # _, fg_mask1 = cv2.threshold(t1_frame_gray, 127, 255, cv2.THRESH_BINARY)
        # _, fg_mask2 = cv2.threshold(t2_frame_gray, 127, 255, cv2.THRESH_BINARY)
        flow = cv2.calcOpticalFlowFarneback(t1_frame_gray, t2_frame_gray, None, 0.5, 3, 50, 3, 5, 1.2, 256)
        nextFlow = cv2.calcOpticalFlowFarneback(t2_frame_gray, frame_gray, None, 0.5, 3, 50, 3, 5, 1.2, 256)
        # flow = cv2.calcOpticalFlowFarneback(fg_mask1, fg_mask2, None, 0.5, 3, 30, 3, 3, 1.1, 0)
        # nextFlow = cv2.calcOpticalFlowFarneback(fg_mask2, fg_mask, None, 0.5, 3, 30, 3, 3, 1.1, 0)
        Flow = nextFlow - flow

        for cX, cY in preCentroids:
            point_st = (cX, cY)
            point_ed = (cX + int(Flow[cY, cX, 0]*100), cY + int(Flow[cY, cX, 1]*100))
            cv2.line(frame_opt, point_st, point_ed, (255, 0, 0), 2)
        preCentroids = centroids.copy()
        writer.write(frame_opt)
        t1_frame = t2_frame.copy()
        t2_frame = frame.copy()
    elif t1_frame is None and t2_frame is not None:
          t1_frame = t2_frame.copy()
          t2_frame = frame.copy()
          writer.write(t2_frame)
          preCentroids = centroids.copy()
    elif t2_frame is None:
      t2_frame = frame.copy()
      writer.write(t2_frame)

cap.release()
writer.release()


# 滑らかなオプティカルフロー

In [ ]:
import cv2
import numpy as np

def compute_optical_flow(image1, image2):
    # 画像の前処理
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Farneback法によるオプティカルフローの計算
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None,
                                        pyr_scale=0.5, levels=3, winsize=15,
                                        iterations=3, poly_n=5, poly_sigma=1.2, flags=0)
    u = flow[..., 0]
    v = flow[..., 1]

    return u, v

# 物体のセグメンテーションと重心計算（仮想的な例）
def segment_and_compute_centroid(image):
    # 簡単な二値化によるセグメンテーション
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

    # 重心の計算
    moments = cv2.moments(binary)
    if moments['m00'] != 0:
        cx = int(moments['m10'] / moments['m00'])
        cy = int(moments['m01'] / moments['m00'])
    else:
        cx, cy = 0, 0

    return cx, cy

# 実行例
image1 = cv2.imread('opticalflow_val/0025.jpg')
image2 = cv2.imread('opticalflow_val/0026.jpg')

u, v = compute_optical_flow(image1, image2)
cx, cy = segment_and_compute_centroid(image1)
width = image1.shape[1]
height = image1.shape[0]
stride_H = 15
stride_W = 15

# 結果の表示
cv2.circle(image1, (cx, cy), 5, (0, 255, 0), -1)
point_st = (cx, cy)
point_ed = (cx + int(u[cy, cx]), cy + int(v[cy, cx]))
cv2.line(image1, point_st, point_ed, (255, 0, 0), 2)

for h in range(0, height, stride_H):
    for w in range(0, width, stride_W):
        point_st = (w, h)
        point_ed = (w + int(u[h, w]*1000), h + int(v[h, w]*1000))
        cv2.line(image1, point_st, point_ed, (0, 0, 255), 2)

# 結果の画像を表示または保存
cv2_imshow(image1)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Optical Flow (u, v):", u, v)
print("Centroid (cx, cy):", cx, cy)


# オプティカルフローの確率的削減

In [ ]:
import cv2
import numpy as np

def compute_optical_flow_at_keypoints(image1, image2, keypoints):
    # 画像の前処理
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # オプティカルフローの計算
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None,
                                        pyr_scale=0.5, levels=3, winsize=15,
                                        iterations=3, poly_n=5, poly_sigma=1.2, flags=0)

    results = []
    for kp in keypoints:
        cx, cy = kp.ravel()
        cx = int(cx)
        cy = int(cy)
        u = flow[cy, cx, 0]
        v = flow[cy, cx, 1]

        # 大きさの計算
        magnitude = np.sqrt(u**2 + v**2)

        # 角度の計算
        angle = np.arctan2(v, u)  # ラジアン
        angle_degrees = np.degrees(angle)  # 度に変換

        results.append((cx, cy, u, v, magnitude, angle, angle_degrees))

    return results

# 物体のセグメンテーションと重心計算（仮想的な例）
def segment_and_compute_centroid(image):
    # 簡単な二値化によるセグメンテーション
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

    # 重心の計算
    moments = cv2.moments(binary)
    if moments['m00'] != 0:
        cx = int(moments['m10'] / moments['m00'])
        cy = int(moments['m01'] / moments['m00'])
    else:
        cx, cy = 0, 0

    return cx, cy

# 特徴点を検出する関数
def detect_keypoints(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    keypoints = cv2.goodFeaturesToTrack(gray, maxCorners=100, qualityLevel=0.01, minDistance=10)
    return keypoints

# 加速度ベクトルを計算する関数
def compute_acceleration(results1, results2, dt):
    accelerations = []
    for (cx1, cy1, u1, v1, mag1, ang1, ang_deg1), (cx2, cy2, u2, v2, mag2, ang2, ang_deg2) in zip(results1, results2):
        # 速度ベクトルの変化量
        du = (u2 - u1) / dt
        dv = (v2 - v1) / dt

        # 加速度ベクトルの大きさ
        accel_magnitude = np.sqrt(du**2 + dv**2)

        # 加速度ベクトルの角度
        accel_angle = np.arctan2(dv, du)
        accel_angle_degrees = np.degrees(accel_angle)

        accelerations.append((cx1, cy1, du, dv, accel_magnitude, accel_angle, accel_angle_degrees))
    return accelerations

# 実行例
image1 = cv2.imread('opticalflow_val/0025.jpg')
image2 = cv2.imread('opticalflow_val/0026.jpg')
image3 = cv2.imread('opticalflow_val/0027.jpg')

# 特徴点を検出
keypoints = detect_keypoints(image1)

# 各特徴点におけるオプティカルフローを計算
results1 = compute_optical_flow_at_keypoints(image1, image2, keypoints)
results2 = compute_optical_flow_at_keypoints(image2, image3, keypoints)

# フレーム間の時間差（秒）
dt = 1 / 60.0  # 例えば、30 FPS の動画の場合

# 加速度ベクトルを計算
accelerations = compute_acceleration(results1, results2, dt)

# 結果の表示
for (cx, cy, u, v, magnitude, angle, angle_degrees) in results1:
    cv2.circle(image1, (cx, cy), 5, (0, 255, 0), -1)
    point_st = (cx, cy)
    point_ed = (cx + int(u)*10, cy + int(v)*10)
    cv2.line(image1, point_st, point_ed, (255, 0, 0), 2)
    print(f"KeyPoint ({cx}, {cy}): u={u}, v={v}, magnitude={magnitude}, angle (radians)={angle}, angle (degrees)={angle_degrees}")

for (cx, cy, u, v, magnitude, angle, angle_degrees) in results2:
    cv2.circle(image2, (cx, cy), 5, (0, 255, 0), -1)
    point_st = (cx, cy)
    point_ed = (cx + int(u)*10, cy + int(v)*10)
    cv2.line(image2, point_st, point_ed, (255, 0, 0), 2)
    print(f"KeyPoint ({cx}, {cy}): u={u}, v={v}, magnitude={magnitude}, angle (radians)={angle}, angle (degrees)={angle_degrees}")

for (cx, cy, du, dv, magnitude, angle, angle_degrees) in accelerations:
    cv2.circle(image2, (cx, cy), 5, (0, 255, 0), -1)
    point_st = (cx, cy)
    point_ed = (cx + int(du)*10, cy + int(dv)*10)
    cv2.line(image2, point_st, point_ed, (0, 0, 255), 2)
    print(f"Acceleration ({cx}, {cy}): du={du}, dv={dv}, magnitude={magnitude}, angle (radians)={angle}, angle (degrees)={angle_degrees}")

# 結果の画像を表示または保存
cv2_imshow(image1)
cv2_imshow(image2)
cv2_imshow(image3)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
image1 = cv2.imread('ball1.jpg')

In [ ]:
width = image1.shape[1]
height = image1.shape[0]
stride_H = 30
stride_W = 30

for h in range(0, height, stride_H):
  for w in range(0, width, stride_W):
    point_st = (w, h)
    point_ed = (w + int(u), h + int(v))
    cv2.line(image1, point_st, point_ed, (0, 0, 255), 2)

cv2_imshow(image1)

In [ ]:
cv2_imshow(image2)

# 最小浮動小数丸め誤差

In [ ]:
# prompt: coding opticalflow

# 最小浮動小数丸め誤差

def compute_optical_flow(image1, image2):
    # 画像の前処理
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # 勾配の計算
    Ix = cv2.Sobel(gray1, cv2.CV_64F, 1, 0, ksize=5)
    Iy = cv2.Sobel(gray1, cv2.CV_64F, 0, 1, ksize=5)
    It = cv2.subtract(gray2, gray1)

    # Lucas-Kanade法による速度ベクトルの計算
    window_size = 5
    half_window = window_size // 2
    u = np.zeros(gray1.shape)
    v = np.zeros(gray1.shape)

    for i in range(half_window, gray1.shape[0] - half_window):
        for j in range(half_window, gray1.shape[1] - half_window):
            Ix_window = Ix[i - half_window:i + half_window + 1, j - half_window:j + half_window + 1].flatten()
            Iy_window = Iy[i - half_window:i + half_window + 1, j - half_window:j + half_window + 1].flatten()
            It_window = It[i - half_window:i + half_window + 1, j - half_window:j + half_window + 1].flatten()

            A = np.vstack((Ix_window, Iy_window)).T
            b = -It_window

            nu = np.linalg.lstsq(A, b, rcond=None)[0]
            u[i, j] = nu[0]
            v[i, j] = nu[1]

    return u, v

# 物体のセグメンテーションと重心計算（仮想的な例）
def segment_and_compute_centroid(image):
    # 簡単な二値化によるセグメンテーション
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

    # 重心の計算
    moments = cv2.moments(binary)
    if moments['m00'] != 0:
        cx = int(moments['m10'] / moments['m00'])
        cy = int(moments['m01'] / moments['m00'])
    else:
        cx, cy = 0, 0

    return cx, cy

# 実行例
image1 = cv2.imread('demo/0001.jpg')
image2 = cv2.imread('demo/0002.jpg')

u, v = compute_optical_flow(image1, image2)
cx, cy = segment_and_compute_centroid(image1)
width = image1.shape[1]
height = image1.shape[0]
stride_H = 15
stride_W = 15

# 結果の表示
cv2.circle(image1, (cx, cy), 5, (0, 255, 0), -1)
point_st = (cx, cy)
point_ed = (cx + int(u[cy,cx]), cy + int(v[cy,cx]))
cv2.line(image1, point_st, point_ed, (255, 0, 0), 2)

for h in range(0, height, stride_H):
  for w in range(0, width, stride_W):
    point_st = (w, h)
    point_ed = (w + int(u[h,w]), h + int(v[h,w]))
    cv2.line(image1, point_st, point_ed, (0, 0, 255), 2)

cv2_imshow(image1)


print("Optical Flow (u, v):", u, v)
print("Centroid (cx, cy):", cx, cy)


In [ ]:
image1 = cv2.imread('ball1.jpg')
image2 = cv2.imread('ball2.jpg')

# u, v = compute_optical_flow(image1, image2)
# cx, cy = segment_and_compute_centroid(image1)

In [ ]:
width = image1.shape[1]
height = image1.shape[0]
stride_H = 10
stride_W = 10

for h in range(0, height, stride_H):
  for w in range(0, width, stride_W):
    point_st = (w, h)
    point_ed = (w + int(u[h,w]*100), h + int(v[h,w]*100))
    cv2.line(image1, point_st, point_ed, (0, 0, 255), 2)

cv2_imshow(image1)

In [ ]:
def total(x):
  total = np.sum(x, axis=(0, 1))
  return total

totalU=total(u)
totalV=total(v)

In [ ]:
point_st = (cx, cy)
point_ed = (cx + int(u[cy,cx]), cy + int(v[cy,cx]))
cv2.line(image1, point_st, point_ed, (255, 0, 0), 2)

# point_st = (cx, cy)
# point_ed = (cx + int(totalU*1000), cy + int(totalV*1000))
# cv2.line(image1, point_st, point_ed, (255, 0, 0), 2)

In [ ]:
cv2_imshow(image1)

In [ ]:
point_st = (cx, cy)
point_ed = (cx + int(u[cy,cx]), cy + int(v[cy,cx]))
cv2.line(image1, point_st, point_ed, (255, 0, 0), 1000)

In [ ]:
cv2_imshow(image1)

In [ ]:
cap = cv2.VideoCapture("outputUniformMotionDemo.mp4")

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

codec = cv2.VideoWriter_fourcc(*'mp4v')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
writer = cv2.VideoWriter('Opticaldemo.avi', codec, fps, (width, height))

while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    frame2 = np.vstack((frame2, bgr))
    writer.write(frame2)
    frame2 = cv2.resize(frame2, None, fx=0.25, fy=0.25, interpolation=cv2.INTER_AREA)
    cv2_imshow(frame2)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    prvs = next
cap.release()
writer.release()
# cv2.destroyAllWindows()

In [ ]:
cap.release()
writer.release()

# 物体の重心を抽出.

# keypointsを加速度ベクトルのみにする.

In [ ]:
import cv2
import numpy as np

def create_gradient_mask(shape, center, radius):
    y, x = np.ogrid[:shape[0], :shape[1]]
    mask = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    mask = np.clip(mask, 0, radius)
    mask = (radius - mask) / radius
    mask = mask * 255  # マスクを0-255の範囲にスケーリング
    return mask.astype(np.uint8)

# def apply_gradient_mask(image, mask):
#     return cv2.multiply(image, mask.astype(np.uint8) / 255.0)
def apply_gradient_mask(image, mask):
    return (image * (mask.astype(np.uint8) / 255.0)).astype(np.uint8)


def compute_optical_flow_at_centroid(image1, image2, cx, cy):
    # 画像の前処理
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # グラデーションマスクの作成
    radius = 4  # マスクの半径
    mask = create_gradient_mask(gray1.shape, (cx, cy), radius)

    # グラデーションマスクを適用
    masked_gray1 = apply_gradient_mask(gray1, mask)
    masked_gray2 = apply_gradient_mask(gray2, mask)

    cv2_imshow(masked_gray1)
    cv2_imshow(masked_gray2)

    # オプティカルフローの計算
    flow = cv2.calcOpticalFlowFarneback(masked_gray1, masked_gray2, None,
                                        pyr_scale=0.5, levels=3, winsize=15,
                                        iterations=3, poly_n=5, poly_sigma=1.2, flags=0)

    u = flow[cy, cx, 0]
    v = flow[cy, cx, 1]

    # 大きさの計算
    magnitude = np.sqrt(u**2 + v**2)

    # 角度の計算
    angle = np.arctan2(v, u)  # ラジアン
    angle_degrees = np.degrees(angle)  # 度に変換

    return (cx, cy, u, v, magnitude, angle, angle_degrees)

# 物体のセグメンテーションと重心計算（仮想的な例）
def segment_and_compute_centroid(image):
    # 簡単な二値化によるセグメンテーション
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

    # 重心の計算
    moments = cv2.moments(binary)
    if moments['m00'] != 0:
        cx = int(moments['m10'] / moments['m00'])
        cy = int(moments['m01'] / moments['m00'])
    else:
        cx, cy = 0, 0

    return cx, cy

# 加速度ベクトルを計算する関数
def compute_acceleration(result1, result2, dt):
    (cx1, cy1, u1, v1, mag1, ang1, ang_deg1) = result1
    (cx2, cy2, u2, v2, mag2, ang2, ang_deg2) = result2

    # 速度ベクトルの変化量
    du = (u2 - u1) / dt
    dv = (v2 - v1) / dt

    # 加速度ベクトルの大きさ
    accel_magnitude = np.sqrt(du**2 + dv**2)

    # 加速度ベクトルの角度
    accel_angle = np.arctan2(dv, du)
    accel_angle_degrees = np.degrees(accel_angle)

    return (cx2, cy2, du, dv, accel_magnitude, accel_angle, accel_angle_degrees)

# 実行例
image1 = cv2.imread('opticalflow_val/0025.jpg')
image2 = cv2.imread('opticalflow_val/0026.jpg')
image3 = cv2.imread('opticalflow_val/0027.jpg')

# 各画像の重心を計算
cx1, cy1 = segment_and_compute_centroid(image1)
cx2, cy2 = segment_and_compute_centroid(image2)
cx3, cy3 = segment_and_compute_centroid(image3)

# 重心におけるオプティカルフローを計算
result1 = compute_optical_flow_at_centroid(image1, image2, cx1, cy1)
result2 = compute_optical_flow_at_centroid(image2, image3, cx2, cy2)

# フレーム間の時間差（秒）
dt = 1 / 60.0  # 例えば、30 FPS の動画の場合

# 加速度ベクトルを計算
acceleration = compute_acceleration(result1, result2, dt)

# 結果の表示
(cx1, cy1, u1, v1, mag1, ang1, ang_deg1) = result1
(cx2, cy2, u2, v2, mag2, ang2, ang_deg2) = result2
(cx3, cy3, du, dv, accel_magnitude, accel_angle, accel_angle_degrees) = acceleration

# image1 にオプティカルフローを描画
cv2.circle(image1, (cx1, cy1), 5, (0, 255, 0), -1)
point_st = (cx1, cy1)
point_ed = (cx1 + int(u1*10**10), cy1 + int(v1*10**10))
cv2.line(image1, point_st, point_ed, (255, 0, 0), 2)
print(f"Frame 1 to Frame 2: KeyPoint ({cx1}, {cy1}): u={u1}, v={v1}, magnitude={mag1}, angle (radians)={ang1}, angle (degrees)={ang_deg1}")

# image2 にオプティカルフローを描画
cv2.circle(image2, (cx2, cy2), 5, (0, 255, 0), -1)
point_st = (cx2, cy2)
point_ed = (cx2 + int(u2*10**12), cy2 + int(v2*10**12))
cv2.line(image2, point_st, point_ed, (255, 0, 0), 2)
print(f"Frame 2 to Frame 3: KeyPoint ({cx2}, {cy2}): u={u2}, v={v2}, magnitude={mag2}, angle (radians)={ang2}, angle (degrees)={ang_deg2}")

# image2 に加速度ベクトルを描画
cv2.circle(image2, (cx2, cy2), 5, (0, 255, 255), -1)  # Yellow circle for acceleration
point_st = (cx2, cy2)
point_ed = (cx2 + int(du*10**10), cy2 + int(dv*10**10))
cv2.line(image2, point_st, point_ed, (0, 0, 255), 2)
print(f"Acceleration: KeyPoint ({cx3}, {cy3}): du={du}, dv={dv}, magnitude={accel_magnitude}, angle (radians)={accel_angle}, angle (degrees)={accel_angle_degrees}")

# 結果の画像を表示または保存
cv2_imshow(image1)
cv2_imshow(image2)
cv2.waitKey(0)
cv2.destroyAllWindows()


## 全フレームに適用

In [ ]:
import cv2
import numpy as np
import glob
import os

def create_gradient_mask(shape, center, radius):
    y, x = np.ogrid[:shape[0], :shape[1]]
    mask = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    mask = np.clip(mask, 0, radius)
    mask = (radius - mask) / radius
    mask = mask * 255  # マスクを0-255の範囲にスケーリング
    return mask.astype(np.uint8)

def apply_gradient_mask(image, mask):
    return (image * (mask.astype(np.float32) / 255.0)).astype(np.uint8)

def compute_optical_flow_at_centroid(image1, image2, cx, cy, prev_cx, prev_cy):
    # 画像の前処理
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # グラデーションマスクの作成
    radius = 50  # マスクの半径
    mask = create_gradient_mask(gray1.shape, (prev_cx, prev_cy), radius)

    # グラデーションマスクを適用
    masked_gray1 = apply_gradient_mask(gray1, mask)
    masked_gray2 = apply_gradient_mask(gray2, mask)

    # オプティカルフローの計算
    flow = cv2.calcOpticalFlowFarneback(masked_gray1, masked_gray2, None,
                                        pyr_scale=0.5, levels=3, winsize=15,
                                        iterations=3, poly_n=5, poly_sigma=1.2, flags=0)

    u = flow[cy, cx, 0]
    v = flow[cy, cx, 1]

    # 大きさの計算
    magnitude = np.sqrt(u**2 + v**2)

    # 角度の計算
    angle = np.arctan2(v, u)  # ラジアン
    angle_degrees = np.degrees(angle)  # 度に変換

    return (cx, cy, u, v, magnitude, angle, angle_degrees)

def segment_and_compute_centroid(image):
    # 簡単な二値化によるセグメンテーション
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

    # 重心の計算
    moments = cv2.moments(binary)
    if moments['m00'] != 0:
        cx = int(moments['m10'] / moments['m00'])
        cy = int(moments['m01'] / moments['m00'])
    else:
        cx, cy = 0, 0

    return cx, cy

def compute_acceleration(result1, result2, dt):
    (cx1, cy1, u1, v1, mag1, ang1, ang_deg1) = result1
    (cx2, cy2, u2, v2, mag2, ang2, ang_deg2) = result2

    # 速度ベクトルの変化量
    du = (u2 - u1) / dt
    dv = (v2 - v1) / dt

    # 加速度ベクトルの大きさ
    accel_magnitude = np.sqrt(du**2 + dv**2)

    # 加速度ベクトルの角度
    accel_angle = np.arctan2(dv, du)
    accel_angle_degrees = np.degrees(accel_angle)

    return (cx2, cy2, du, dv, accel_magnitude, accel_angle, accel_angle_degrees)

# 画像が格納されているフォルダのパス
image_folder = 'newmethod'

# 画像ファイルをすべて取得
image_files = sorted(glob.glob(os.path.join(image_folder, '*.png')))

# フレーム間の時間差（秒）
dt = 1 / 30.0  # 例えば、30 FPS の動画の場合

# 初期フレームの重心を計算
initial_frame = cv2.imread(image_files[0])
prev_cx, prev_cy = segment_and_compute_centroid(initial_frame)

# すべてのフレームを処理
for i in range(len(image_files) - 2):
    image1 = cv2.imread(image_files[i])
    image2 = cv2.imread(image_files[i + 1])
    image3 = cv2.imread(image_files[i + 2])

    # 各画像の重心を計算
    cx1, cy1 = segment_and_compute_centroid(image1)
    cx2, cy2 = segment_and_compute_centroid(image2)
    cx3, cy3 = segment_and_compute_centroid(image3)

    # 重心におけるオプティカルフローを計算
    result1 = compute_optical_flow_at_centroid(image1, image2, cx1, cy1, prev_cx, prev_cy)
    result2 = compute_optical_flow_at_centroid(image2, image3, cx2, cy2, cx1, cy1)

    # 加速度ベクトルを計算
    acceleration = compute_acceleration(result1, result2, dt)

    # 結果の表示
    (cx1, cy1, u1, v1, mag1, ang1, ang_deg1) = result1
    (cx2, cy2, u2, v2, mag2, ang2, ang_deg2) = result2
    (cx3, cy3, du, dv, accel_magnitude, accel_angle, accel_angle_degrees) = acceleration

    # image1 にオプティカルフローを描画
    cv2.circle(image1, (cx1, cy1), 5, (0, 255, 0), -1)
    point_st = (cx1, cy1)
    point_ed = (cx1 + int(u1 * 10 ** 7), cy1 + int(v1 * 10 ** 7))  # スケールを適切に調整
    cv2.line(image1, point_st, point_ed, (255, 0, 0), 2)
    print(f"Frame {i+1} to Frame {i+2}: KeyPoint ({cx1}, {cy1}): u={u1}, v={v1}, magnitude={mag1}, angle (radians)={ang1}, angle (degrees)={ang_deg1}")

    # image2 にオプティカルフローを描画
    cv2.circle(image2, (cx2, cy2), 5, (0, 255, 0), -1)
    point_st = (cx2, cy2)
    point_ed = (cx2 + int(u2 * 10 ** 7), cy2 + int(v2 * 10 ** 7))  # スケールを適切に調整
    cv2.line(image2, point_st, point_ed, (255, 0, 0), 2)
    print(f"Frame {i+2} to Frame {i+3}: KeyPoint ({cx2}, {cy2}): u={u2}, v={v2}, magnitude={mag2}, angle (radians)={ang2}, angle (degrees)={ang_deg2}")

    # image2 に加速度ベクトルを描画
    cv2.circle(image2, (cx2, cy2), 5, (0, 255, 255), -1)  # Yellow circle for acceleration
    point_st = (cx2, cy2)
    point_ed = (cx2 + int(du * 10 ** 6), cy2 + int(dv * 10 ** 6))  # スケールを適切に調整
    cv2.line(image2, point_st, point_ed, (0, 0, 255), 2)
    print(f"Acceleration: KeyPoint ({cx3}, {cy3}): du={du}, dv={dv}, magnitude={accel_magnitude}, angle (radians)={accel_angle}, angle (degrees)={accel_angle_degrees}")

    # 結果の画像を表示または保存
    # print(f'Result Frame {i+1}')
    # cv2_imshow(image1)
    print(f'Result Frame {i+2}')
    cv2_imshow(image2)
    cv2.waitKey(0)

    # 次のフレームのために重心を更新
    prev_cx, prev_cy = cx1, cy1

cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# シミュレーションパラメータ
num_frames = 60
frame_size = (200, 200)
radius = 50
center = (100, 100)
omega = 2 * np.pi / num_frames  # 一周するための角速度

# フレームの生成
frames = []
for t in range(num_frames):
    frame = np.zeros(frame_size, dtype=np.uint8)
    x = int(center[0] + radius * np.cos(omega * t))
    y = int(center[1] + radius * np.sin(omega * t))
    cv2.circle(frame, (x, y), 5, 255, -1)
    frames.append(frame)

# 重心の検出と追跡
lk_params = dict(winSize=(5, 5), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
p0 = np.array([[center]], dtype=np.float32)  # 初期位置を重心に設定

# 特徴点の追跡
for i in range(1, num_frames):
    p1, st, err = cv2.calcOpticalFlowPyrLK(frames[i-1], frames[i], p0, None, **lk_params)
    if p1 is not None:
        p0 = p1
    else:
        p0 = np.array([[center]], dtype=np.float32)  # トラッキングが失敗した場合、中心に戻す

    # ベクトルの表示
    vis = cv2.cvtColor(frames[i], cv2.COLOR_GRAY2BGR)
    for pt in p1:
        x, y = pt.ravel()
        cv2.circle(vis, (int(x), int(y)), 5, (0, 255, 0), -1)

    print('Feature Tracking')
    cv2_imshow(vis)
    cv2.waitKey(50)

cv2.destroyAllWindows()


# 円運動を行う物体の加速度オプティカルフロー

In [ ]:
import cv2
import numpy as np

# シミュレーションパラメータ
num_frames = 60
frame_size = (200, 200)
radius = 50
center = (100, 100)
omega = 2 * np.pi / num_frames  # 一周するための角速度

# フレームの生成
frames = []
for t in range(num_frames):
    frame = np.zeros(frame_size, dtype=np.uint8)
    x = int(center[0] + radius * np.cos(omega * t))
    y = int(center[1] + radius * np.sin(omega * t))
    cv2.circle(frame, (x, y), 5, 255, -1)
    frames.append(frame)

# フレームの表示（確認用）
for frame in frames:
    print('Circular Motion')
    cv2_imshow(frame)
    cv2.waitKey(100)

cv2.destroyAllWindows()


In [ ]:
# オプティカルフローの計算と可視化
for i in range(num_frames - 1):
    frame1 = frames[i]
    frame2 = frames[i + 1]

    # オプティカルフローの計算
    flow = cv2.calcOpticalFlowFarneback(frame1, frame2, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # 結果の可視化
    hsv = np.zeros_like(cv2.cvtColor(frame1, cv2.COLOR_GRAY2BGR))
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 1] = 255
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    print('Optical Flow')
    cv2_imshow(rgb)
    cv2.waitKey(100)

cv2.destroyAllWindows()


In [ ]:
# 加速度の計算と可視化
accelerations = []
for i in range(num_frames - 2):
    frame1 = frames[i]
    frame2 = frames[i + 1]
    frame3 = frames[i + 2]

    # 速度ベクトルの計算
    flow1 = cv2.calcOpticalFlowFarneback(frame1, frame2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    flow2 = cv2.calcOpticalFlowFarneback(frame2, frame3, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # 加速度ベクトルの計算
    acceleration = flow2 - flow1
    accelerations.append(acceleration)

    # 結果の可視化
    hsv = np.zeros_like(cv2.cvtColor(frame1, cv2.COLOR_GRAY2BGR))
    mag, ang = cv2.cartToPolar(acceleration[..., 0], acceleration[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 1] = 255
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    print(i)
    cv2_imshow(rgb)
    cv2.waitKey(100)

cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# シミュレーションパラメータ
num_frames = 60
frame_size = (200, 200)
radius = 50
center = (100, 100)
omega = 2 * np.pi / num_frames  # 一周するための角速度

# フレームの生成
frames = []
for t in range(num_frames):
    frame = np.zeros(frame_size, dtype=np.uint8)
    x = int(center[0] + radius * np.cos(omega * t))
    y = int(center[1] + radius * np.sin(omega * t))
    cv2.circle(frame, (x, y), 5, 255, -1)
    frames.append(frame)

# フレームの表示（確認用）
for frame in frames:
    cv2_imshow(frame)
    cv2.waitKey(100)

cv2.destroyAllWindows()


In [ ]:
# オプティカルフローの計算と可視化
for i in range(num_frames - 1):
    frame1 = frames[i]
    frame2 = frames[i + 1]

    # オプティカルフローの計算
    flow = cv2.calcOpticalFlowFarneback(frame1, frame2, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # 結果の可視化
    hsv = np.zeros_like(cv2.cvtColor(frame1, cv2.COLOR_GRAY2BGR))
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 1] = 255
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    cv2_imshow(rgb)
    cv2.waitKey(100)

cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# シミュレーションパラメータ
num_frames = 120  # フレーム数を増やしてフレーム間の変位を小さくする
frame_size = (200, 200)
radius = 50
center = (100, 100)
omega = 2 * np.pi / num_frames  # 一周するための角速度

# フレームの生成
frames = []
for t in range(num_frames):
    frame = np.zeros(frame_size, dtype=np.uint8)
    x = int(center[0] + radius * np.cos(omega * t))
    y = int(center[1] + radius * np.sin(omega * t))
    cv2.circle(frame, (x, y), 5, 255, -1)
    frames.append(frame)

# フレームの表示（確認用）
for frame in frames:
    cv2_imshow(frame)
    cv2.waitKey(50)  # フレームレートを上げる

# オプティカルフローの計算とベクトル表示
for i in range(num_frames - 1):
    frame1 = frames[i]
    frame2 = frames[i + 1]

    # オプティカルフローの計算
    flow = cv2.calcOpticalFlowFarneback(frame1, frame2, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # ベクトルの表示
    h, w = frame1.shape
    step = 16
    y, x = np.mgrid[step/2:h:step, step/2:w:step].astype(np.int64)
    fx, fy = flow[y, x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = cv2.cvtColor(frame1, cv2.COLOR_GRAY2BGR)
    cv2.polylines(vis, lines, 0, (0, 255, 0))

    for (x1, y1), (x2, y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)

    cv2_imshow(vis)
    cv2.waitKey(50)

cv2.destroyAllWindows()

# 特徴点の検出と追跡
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# 初期フレームの特徴点を検出
p0 = cv2.goodFeaturesToTrack(frames[0], mask=None, **feature_params)

# 特徴点の追跡
for i in range(1, num_frames):
    p1, st, err = cv2.calcOpticalFlowPyrLK(frames[i-1], frames[i], p0, None, **lk_params)

    # ベクトルの表示
    vis = cv2.cvtColor(frames[i], cv2.COLOR_GRAY2BGR)
    for pt in p1:
        x, y = pt.ravel()
        cv2.circle(vis, (x, y), 5, (0, 255, 0), -1)

    cv2_imshow(vis)
    cv2.waitKey(50)

cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

def create_gradient_mask(shape, center, radius):
    y, x = np.ogrid[:shape[0], :shape[1]]
    mask = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    mask = np.clip(mask, 0, radius)
    mask = (radius - mask) / radius
    mask = mask * 255  # マスクを0-255の範囲にスケーリング
    return mask.astype(np.uint8)

def apply_gradient_mask(image, mask):
    return (image * (mask.astype(np.float32) / 255.0)).astype(np.uint8)

def compute_optical_flow_at_centroid(image1, image2, cx, cy):
    # 画像の前処理
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # グラデーションマスクの作成
    radius = 50  # マスクの半径
    mask = create_gradient_mask(gray1.shape, (cx, cy), radius)

    # グラデーションマスクを適用
    masked_gray1 = apply_gradient_mask(gray1, mask)
    masked_gray2 = apply_gradient_mask(gray2, mask)

    # オプティカルフローの計算
    flow = cv2.calcOpticalFlowFarneback(masked_gray1, masked_gray2, None,
                                        pyr_scale=0.5, levels=3, winsize=15,
                                        iterations=3, poly_n=5, poly_sigma=1.2, flags=0)

    # サブピクセル精度での流れを取得
    u = cv2.getRectSubPix(flow[..., 0], (1, 1), (float(cx), float(cy)))[0, 0]
    v = cv2.getRectSubPix(flow[..., 1], (1, 1), (float(cx), float(cy)))[0, 0]

    # 大きさの計算
    magnitude = np.sqrt(u**2 + v**2)

    # 角度の計算
    angle = np.arctan2(v, u)  # ラジアン
    angle_degrees = np.degrees(angle)  # 度に変換

    return (cx, cy, u, v, magnitude, angle, angle_degrees)

def segment_and_compute_centroid(image):
    # 簡単な二値化によるセグメンテーション
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

    # 重心の計算
    moments = cv2.moments(binary)
    if moments['m00'] != 0:
        cx = moments['m10'] / moments['m00']
        cy = moments['m01'] / moments['m00']
    else:
        cx, cy = 0.0, 0.0

    return cx, cy

def compute_acceleration(result1, result2, dt):
    (cx1, cy1, u1, v1, mag1, ang1, ang_deg1) = result1
    (cx2, cy2, u2, v2, mag2, ang2, ang_deg2) = result2

    # 速度ベクトルの変化量
    du = (u2 - u1) / dt
    dv = (v2 - v1) / dt

    # 加速度ベクトルの大きさ
    accel_magnitude = np.sqrt(du**2 + dv**2)

    # 加速度ベクトルの角度
    accel_angle = np.arctan2(dv, du)
    accel_angle_degrees = np.degrees(accel_angle)

    return (cx2, cy2, du, dv, accel_magnitude, accel_angle, accel_angle_degrees)

# 画像が格納されているフォルダのパス
image_folder = 'newmethod'

# 画像ファイルをすべて取得
image_files = sorted(glob.glob(os.path.join(image_folder, '*.png')))

# フレーム間の時間差（秒）
dt = 1 / 30.0  # 例えば、30 FPS の動画の場合

# 初期フレームの重心を計算
initial_frame = cv2.imread(image_files[0])
prev_cx, prev_cy = segment_and_compute_centroid(initial_frame)

# すべてのフレームを処理
for i in range(len(image_files) - 2):
    image1 = cv2.imread(image_files[i])
    image2 = cv2.imread(image_files[i + 1])
    image3 = cv2.imread(image_files[i + 2])

    # 各画像の重心を計算
    cx1, cy1 = segment_and_compute_centroid(image1)
    cx2, cy2 = segment_and_compute_centroid(image2)
    cx3, cy3 = segment_and_compute_centroid(image3)

    # 重心におけるオプティカルフローを計算
    result1 = compute_optical_flow_at_centroid(image1, image2, cx1, cy1)
    result2 = compute_optical_flow_at_centroid(image2, image3, cx2, cy2)

    # 加速度ベクトルを計算
    acceleration = compute_acceleration(result1, result2, dt)

    # 結果の表示
    (cx1, cy1, u1, v1, mag1, ang1, ang_deg1) = result1
    (cx2, cy2, u2, v2, mag2, ang2, ang_deg2) = result2
    (cx3, cy3, du, dv, accel_magnitude, accel_angle, accel_angle_degrees) = acceleration

    # image1 にオプティカルフローを描画
    cv2.circle(image1, (int(cx1), int(cy1)), 5, (0, 255, 0), -1)
    point_st = (int(cx1), int(cy1))
    point_ed = (int(cx1 + u1 * 10), int(cy1 + v1 * 10))  # スケールを適切に調整
    cv2.line(image1, point_st, point_ed, (255, 0, 0), 2)
    print(f"Frame {i+1} to Frame {i+2}: KeyPoint ({cx1}, {cy1}): u={u1}, v={v1}, magnitude={mag1}, angle (radians)={ang1}, angle (degrees)={ang_deg1}")

    # image2 にオプティカルフローを描画
    cv2.circle(image2, (int(cx2), int(cy2)), 5, (0, 255, 0), -1)
    point_st = (int(cx2), int(cy2))
    point_ed = (int(cx2 + u2 * 10), int(cy2 + v2 * 10))  # スケールを適切に調整
    cv2.line(image2, point_st, point_ed, (255, 0, 0), 2)
    print(f"Frame {i+2} to Frame {i+3}: KeyPoint ({cx2}, {cy2}): u={u2}, v={v2}, magnitude={mag2}, angle (radians)={ang2}, angle (degrees)={ang_deg2}")

    # image2 に加速度ベクトルを描画
    cv2.circle(image2, (int(cx2), int(cy2)), 5, (0, 255, 255), -1)  # Yellow circle for acceleration
    point_st = (int(cx2), int(cy2))
    point_ed = (int(cx2 + du * 10), int(cy2 + dv * 10))  # スケールを適切に調整
    cv2.line(image2, point_st, point_ed, (0, 0, 255), 2)
    print(f"Acceleration: KeyPoint ({cx3}, {cy3}): du={du}, dv={dv}, magnitude={accel_magnitude}, angle (radians)={accel_angle}, angle (degrees)={accel_angle_degrees}")

    # 結果の画像を表示または保存
    # print(f'Result Frame {i+1}')
    # cv2_imshow(image1)
    print(f'Result Frame {i+2}')
    cv2_imshow(image2)
    cv2.waitKey(0)

    # 次のフレームのために重心を更新
    prev_cx, prev_cy = cx2, cy2

cv2.destroyAllWindows()



# 重心座標を浮動小数点として計算.

In [ ]:
import cv2
import numpy as np
import glob
import os
import pandas as pd

def create_gradient_mask(shape, center, radius):
    y, x = np.ogrid[:shape[0], :shape[1]]
    mask = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    mask = np.clip(mask, 0, radius)
    mask = (radius - mask) / radius
    mask = mask * 255  # マスクを0-255の範囲にスケーリング
    return mask.astype(np.uint8)

def apply_gradient_mask(image, mask):
    return (image * (mask.astype(np.float32) / 255.0)).astype(np.uint8)

def compute_optical_flow_at_centroid(image1, image2, cx, cy):
    # 画像の前処理
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # グラデーションマスクの作成
    radius = 50  # マスクの半径
    mask = create_gradient_mask(gray1.shape, (cx, cy), radius)

    # グラデーションマスクを適用
    masked_gray1 = apply_gradient_mask(gray1, mask)
    masked_gray2 = apply_gradient_mask(gray2, mask)

    # オプティカルフローの計算
    flow = cv2.calcOpticalFlowFarneback(masked_gray1, masked_gray2, None,
                                        pyr_scale=0.5, levels=3, winsize=15,
                                        iterations=3, poly_n=5, poly_sigma=1.2, flags=0)

    # サブピクセル精度での流れを取得
    u = cv2.getRectSubPix(flow[..., 0], (1, 1), (float(cx), float(cy)))[0, 0]
    v = cv2.getRectSubPix(flow[..., 1], (1, 1), (float(cx), float(cy)))[0, 0]

    # 大きさの計算
    magnitude = np.sqrt(u**2 + v**2)

    # 角度の計算
    angle = np.arctan2(v, u)  # ラジアン
    angle_degrees = np.degrees(angle)  # 度に変換

    return (cx, cy, u, v, magnitude, angle, angle_degrees)

def segment_and_compute_centroid(image):
    # 簡単な二値化によるセグメンテーション
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

    # 重心の計算
    moments = cv2.moments(binary)
    if moments['m00'] != 0:
        cx = moments['m10'] / moments['m00']
        cy = moments['m01'] / moments['m00']
    else:
        cx, cy = 0.0, 0.0

    return cx, cy

def compute_acceleration(result1, result2, dt):
    (cx1, cy1, u1, v1, mag1, ang1, ang_deg1) = result1
    (cx2, cy2, u2, v2, mag2, ang2, ang_deg2) = result2

    # 速度ベクトルの変化量
    du = (u2 - u1) / dt
    dv = (v2 - v1) / dt

    # 加速度ベクトルの大きさ
    accel_magnitude = np.sqrt(du**2 + dv**2)

    # 加速度ベクトルの角度
    accel_angle = np.arctan2(dv, du)
    accel_angle_degrees = np.degrees(accel_angle)

    return (cx2, cy2, du, dv, accel_magnitude, accel_angle, accel_angle_degrees)

def theoretical_values(fr, r, v):
    w = v / r
    x = r * np.cos(w * fr)
    y = r * np.sin(w * fr)
    Vx = -v * np.sin(w * fr)
    Vy = v * np.cos(w * fr)
    Ax = -v * w * np.cos(w * fr)
    Ay = -v * w * np.sin(w * fr)
    return x, y, Vx, Vy, Ax, Ay

# 画像が格納されているフォルダのパス
image_folder = 'newmethod'

# 画像ファイルをすべて取得
image_files = sorted(glob.glob(os.path.join(image_folder, '*.png')))

# フレーム間の時間差（秒）
dt = 1 / 30.0  # 例えば、30 FPS の動画の場合

# 初期フレームの重心を計算
initial_frame = cv2.imread(image_files[0])
prev_cx, prev_cy = segment_and_compute_centroid(initial_frame)

# 結果を格納するリスト
results = []

# すべてのフレームを処理
for i in range(len(image_files) - 2):
    image1 = cv2.imread(image_files[i])
    image2 = cv2.imread(image_files[i + 1])
    image3 = cv2.imread(image_files[i + 2])

    # 各画像の重心を計算
    cx1, cy1 = segment_and_compute_centroid(image1)
    cx2, cy2 = segment_and_compute_centroid(image2)
    cx3, cy3 = segment_and_compute_centroid(image3)

    # 重心におけるオプティカルフローを計算
    result1 = compute_optical_flow_at_centroid(image1, image2, cx1, cy1)
    result2 = compute_optical_flow_at_centroid(image2, image3, cx2, cy2)

    # 加速度ベクトルを計算
    acceleration = compute_acceleration(result1, result2, dt)

    # 理論値を計算
    theoX, theoY, theoVx, theoVy, theoAx, theoAy = theoretical_values(i+1, 200, 1.0)

    # 結果をリストに追加
    results.append([i+1, cx1, cy1, result1[2], result1[3], acceleration[2], acceleration[3],
                    theoX, theoY, theoVx, theoVy, theoAx, theoAy])

    # 次のフレームのために重心を更新
    prev_cx, prev_cy = cx2, cy2

# 結果をデータフレームに変換してCSVファイルに保存
df = pd.DataFrame(results, columns=['Frame', 'X', 'Y', 'Vx', 'Vy', 'Ax', 'Ay', 'TheoX', 'TheoY', 'TheoVx', 'TheoVy', 'TheoAx', 'TheoAy'])
df.to_csv('optical_flow_results.csv', index=False)

print("結果をoptical_flow_results.csvに保存しました。")


# 重心と座標を用いて加速度オプティカルフローを求める.

In [ ]:
import cv2
import numpy as np
import glob
import os
import pandas as pd

def segment_and_compute_centroid(image):
    # 簡単な二値化によるセグメンテーション
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

    # 重心の計算
    moments = cv2.moments(binary)
    if moments['m00'] != 0:
        cx = moments['m10'] / moments['m00']
        cy = moments['m01'] / moments['m00']
    else:
        cx, cy = 0.0, 0.0

    return cx, cy

def compute_velocity_and_acceleration(centroid1, centroid2, centroid3, dt):
    cx1, cy1 = centroid1
    cx2, cy2 = centroid2
    cx3, cy3 = centroid3

    # 速度の計算
    Vx1 = (cx2 - cx1) / dt
    Vy1 = (cy2 - cy1) / dt
    Vx2 = (cx3 - cx2) / dt
    Vy2 = (cy3 - cy2) / dt

    # 加速度の計算
    Ax = (Vx2 - Vx1) / dt
    Ay = (Vy2 - Vy1) / dt

    return Vx1, Vy1, Ax, Ay

def theoretical_values(fr, r, v):
    w = v / r
    x = r * np.cos(w * fr)
    y = r * np.sin(w * fr)
    Vx = -v * np.sin(w * fr)
    Vy = v * np.cos(w * fr)
    Ax = -v * w * np.cos(w * fr)
    Ay = -v * w * np.sin(w * fr)
    return x, y, Vx, Vy, Ax, Ay

# 画像が格納されているフォルダのパス
image_folder = 'newmethod'

# 画像ファイルをすべて取得
image_files = sorted(glob.glob(os.path.join(image_folder, '*.png')))

# フレーム間の時間差（秒）
dt = 1 / 30.0  # 例えば、30 FPS の動画の場合

# 結果を格納するリスト
results = []

# すべてのフレームを処理
for i in range(len(image_files) - 2):
    image1 = cv2.imread(image_files[i])
    image2 = cv2.imread(image_files[i + 1])
    image3 = cv2.imread(image_files[i + 2])

    # 各画像の重心を計算
    cx1, cy1 = segment_and_compute_centroid(image1)
    cx2, cy2 = segment_and_compute_centroid(image2)
    cx3, cy3 = segment_and_compute_centroid(image3)

    # 重心の座標から速度と加速度を計算
    Vx, Vy, Ax, Ay = compute_velocity_and_acceleration((cx1, cy1), (cx2, cy2), (cx3, cy3), dt)

    # 理論値を計算
    theoX, theoY, theoVx, theoVy, theoAx, theoAy = theoretical_values(i+1, 200, 1.0)

    # 結果をリストに追加
    results.append([i+1, cx1, cy1, Vx, Vy, Ax, Ay, theoX, theoY, theoVx, theoVy, theoAx, theoAy])

# 結果をデータフレームに変換してCSVファイルに保存
df = pd.DataFrame(results, columns=['Frame', 'X', 'Y', 'Vx', 'Vy', 'Ax', 'Ay', 'TheoX', 'TheoY', 'TheoVx', 'TheoVy', 'TheoAx', 'TheoAy'])
df.to_csv('centroid_based_acceleration.csv', index=False)

print("結果をcentroid_based_acceleration.csvに保存しました。")


In [ ]:
import cv2
import numpy as np
import glob
import os
import pandas as pd

def segment_and_compute_centroid(image):
    # 簡単な二値化によるセグメンテーション
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

    # 重心の計算
    moments = cv2.moments(binary)
    if moments['m00'] != 0:
        cx = moments['m10'] / moments['m00']
        cy = moments['m01'] / moments['m00']
    else:
        cx, cy = 0.0, 0.0

    return cx, cy

def compute_velocity_and_acceleration(centroid1, centroid2, centroid3, dt):
    cx1, cy1 = centroid1
    cx2, cy2 = centroid2
    cx3, cy3 = centroid3

    # 速度の計算
    Vx1 = (cx2 - cx1) / dt
    Vy1 = (cy2 - cy1) / dt
    Vx2 = (cx3 - cx2) / dt
    Vy2 = (cy3 - cy2) / dt

    # 加速度の計算
    Ax = (Vx2 - Vx1) / dt
    Ay = (Vy2 - Vy1) / dt

    return Vx1, Vy1, Ax, Ay

def theoretical_values(fr, r, v):
    w = v / r
    x = r * np.cos(w * fr)
    y = r * np.sin(w * fr)
    Vx = -v * np.sin(w * fr)
    Vy = v * np.cos(w * fr)
    Ax = -v * w * np.cos(w * fr)
    Ay = -v * w * np.sin(w * fr)
    return x, y, Vx, Vy, Ax, Ay

# 画像が格納されているフォルダのパス
image_folder = 'newmethod'

# 画像ファイルをすべて取得
image_files = sorted(glob.glob(os.path.join(image_folder, '*.png')))

# フレーム間の時間差（秒）
dt = 1 / 30.0  # 例えば、30 FPS の動画の場合

# 結果を格納するリスト
results = []

# すべてのフレームを処理
for i in range(len(image_files) - 2):
    image1 = cv2.imread(image_files[i])
    image2 = cv2.imread(image_files[i + 1])
    image3 = cv2.imread(image_files[i + 2])

    # 各画像の重心を計算
    cx1, cy1 = segment_and_compute_centroid(image1)
    cx2, cy2 = segment_and_compute_centroid(image2)
    cx3, cy3 = segment_and_compute_centroid(image3)

    # 重心の座標から速度と加速度を計算
    Vx, Vy, Ax, Ay = compute_velocity_and_acceleration((cx1, cy1), (cx2, cy2), (cx3, cy3), dt)

    # 加速度ベクトルを描画
    start_point = (int(cx2), int(cy2))
    end_point = (int(cx2 + Ax * 10), int(cy2 + Ay * 10))
    cv2.arrowedLine(image2, start_point, end_point, (0, 0, 255), 2)

    # 結果をリストに追加
    results.append([i+1, cx1, cy1, Vx, Vy, Ax, Ay])

    # 描画されたフレームを保存
    cv2.imwrite(f'output_frame_{i+1}.png', image2)

# 結果をデータフレームに変換してCSVファイルに保存
df = pd.DataFrame(results, columns=['Frame', 'X', 'Y', 'Vx', 'Vy', 'Ax', 'Ay'])
df.to_csv('centroid_based_acceleration.csv', index=False)

print("結果をcentroid_based_acceleration.csvに保存し、フレームを保存しました。")


In [ ]:
import cv2
import numpy as np
import glob
import os
import pandas as pd
from scipy.signal import savgol_filter

def segment_and_compute_centroid(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    moments = cv2.moments(binary)
    if moments['m00'] != 0:
        cx = moments['m10'] / moments['m00']
        cy = moments['m01'] / moments['m00']
    else:
        cx, cy = 0.0, 0.0
    return cx, cy

def compute_velocity_and_acceleration(centroid1, centroid2, centroid3, dt):
    cx1, cy1 = centroid1
    cx2, cy2 = centroid2
    cx3, cy3 = centroid3
    # Vx1 = (cx2 - cx1) / dt
    # Vy1 = (cy2 - cy1) / dt
    # Vx2 = (cx3 - cx2) / dt
    # Vy2 = (cy3 - cy2) / dt
    # Ax = (Vx2 - Vx1) / dt
    # Ay = (Vy2 - Vy1) / dt
    Vx1 = (cx2 - cx1)
    Vy1 = (cy2 - cy1)
    Vx2 = (cx3 - cx2)
    Vy2 = (cy3 - cy2)
    Ax = (Vx2 - Vx1)
    Ay = (Vy2 - Vy1)
    return Vx1, Vy1, Ax, Ay

def theoretical_values(fr, r, v):
    w = v / r
    x = r * np.cos(w * fr)
    y = r * np.sin(w * fr)
    Vx = -v * np.sin(w * fr)
    Vy = v * np.cos(w * fr)
    Ax = -v * w * np.cos(w * fr)
    Ay = -v * w * np.sin(w * fr)
    return x, y, Vx, Vy, Ax, Ay

image_folder = 'newmethod'
image_files = sorted(glob.glob(os.path.join(image_folder, '*.png')))
dt = 1 / 30.0

results = []
for i in range(len(image_files) - 2):
    image1 = cv2.imread(image_files[i])
    image2 = cv2.imread(image_files[i + 1])
    image3 = cv2.imread(image_files[i + 2])
    cx1, cy1 = segment_and_compute_centroid(image1)
    cx2, cy2 = segment_and_compute_centroid(image2)
    cx3, cy3 = segment_and_compute_centroid(image3)
    Vx, Vy, Ax, Ay = compute_velocity_and_acceleration((cx1, cy1), (cx2, cy2), (cx3, cy3), dt)
    results.append([i+1, cx1, cy1, Vx, Vy, Ax, Ay])

# データを平滑化
df = pd.DataFrame(results, columns=['Frame', 'X', 'Y', 'Vx', 'Vy', 'Ax', 'Ay'])
df['Vx_smooth'] = savgol_filter(df['Vx'], window_length=5, polyorder=2)
df['Vy_smooth'] = savgol_filter(df['Vy'], window_length=5, polyorder=2)

# 平滑化後の速度から加速度を再計算
df['Ax_smooth'] = df['Vx_smooth'].diff()
df['Ay_smooth'] = df['Vy_smooth'].diff()
# NaN値を0に置換
df.fillna(0, inplace=True)
# 各フレームに加速度ベクトルを描画
for i in range(len(image_files) - 2):
    image2 = cv2.imread(image_files[i + 1])
    cx2, cy2 = df.loc[i, ['X', 'Y']]
    Ax, Ay = df.loc[i, ['Ax_smooth', 'Ay_smooth']]
    start_point = (int(cx2), int(cy2))
    end_point = (int(cx2 + Ax * 10), int(cy2 + Ay * 10))
    cv2.arrowedLine(image2, start_point, end_point, (0, 0, 255), 2)
    cv2.imwrite(f'outputV2_frame_{i+1}.png', image2)

df.to_csv('centroid_based_acceleration_smoothV3.csv', index=False)
print("結果をcentroid_based_acceleration_smooth.csvに保存し、フレームを保存しました。")


In [ ]:
df.to_csv('centroid_based_acceleration_smoothV2.csv', index=False)

In [ ]:
!pip install filterpy pandas

In [ ]:
import cv2
import numpy as np
import glob
import os
import pandas as pd
from scipy.signal import savgol_filter, butter, filtfilt
from scipy.ndimage import gaussian_filter1d
from filterpy.kalman import KalmanFilter

def initialize_kalman_filter():
    """

    Returns:

    """
    kf = KalmanFilter(dim_x=2, dim_z=1)
    kf.F = np.array([[1, 1], [0, 1]])  # 状態遷移行列
    kf.H = np.array([[1, 0]])          # 観測行列
    kf.P *= 1000.                      # 誤差共分散行列の初期設定
    kf.R = np.array([[5]])             # 観測ノイズの共分散
    kf.Q = np.array([[1, 0], [0, 1]])  # プロセスノイズの共分散
    kf.x = np.array([[0], [0]])        # 初期状態
    return kf

def kalman_filter_acceleration(data):
    """

    Args:
      data:

    Returns:

    """
    kf = initialize_kalman_filter()
    filtered_data = []
    for measurement in data:
        kf.predict()
        kf.update([measurement])
        filtered_data.append(kf.x[0][0])
    return np.array(filtered_data)

def segment_and_compute_centroid(image):
    """

    Args:
      image:

    Returns:

    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    moments = cv2.moments(binary)
    if moments['m00'] != 0:
        cx = moments['m10'] / moments['m00']
        cy = moments['m01'] / moments['m00']
    else:
        cx, cy = 0.0, 0.0
    return cx, cy

def compute_velocity_and_acceleration(centroid1, centroid2, centroid3, dt):
    """

    Args:
      centroid1:
      centroid2:
      centroid3:
      dt:

    Returns:

    """
    cx1, cy1 = centroid1
    cx2, cy2 = centroid2
    cx3, cy3 = centroid3

    # Vx1 = (cx2 - cx1) / dt
    # Vy1 = (cy2 - cy1) / dt
    # Vx2 = (cx3 - cx2) / dt
    # Vy2 = (cy3 - cy2) / dt

    # Ax = (Vx2 - Vx1) / dt
    # Ay = (Vy2 - Vy1) / dt

    Vx1 = (cx2 - cx1)
    Vy1 = (cy2 - cy1)
    Vx2 = (cx3 - cx2)
    Vy2 = (cy3 - cy2)

    Ax = (Vx2 - Vx1)
    Ay = (Vy2 - Vy1)

    return Vx1, Vy1, Ax, Ay

def theoretical_values(fr, r, v):
    """

    Args:
      fr:
      r:
      v:

    Returns:

    """
    w = v / r
    x = r * np.cos(w * fr)
    y = r * np.sin(w * fr)
    Vx = -v * np.sin(w * fr)
    Vy = v * np.cos(w * fr)
    Ax = -v * w * np.cos(w * fr)
    Ay = -v * w * np.sin(w * fr)
    return x, y, Vx, Vy, Ax, Ay

def butter_lowpass_filter(data, cutoff, fs, order=5):
    """

    Args:
      data:
      cutoff:
      fs:
      order:

    Returns:

    """
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

image_folder = 'newmethod'
image_files = sorted(glob.glob(os.path.join(image_folder, '*.png')))
dt = 1 / 30.0

results = []
for i in range(len(image_files) - 2):
    image1 = cv2.imread(image_files[i])
    image2 = cv2.imread(image_files[i + 1])
    image3 = cv2.imread(image_files[i + 2])
    cx1, cy1 = segment_and_compute_centroid(image1)
    cx2, cy2 = segment_and_compute_centroid(image2)
    cx3, cy3 = segment_and_compute_centroid(image3)
    Vx, Vy, Ax, Ay = compute_velocity_and_acceleration((cx1, cy1), (cx2, cy2), (cx3, cy3), dt)
    results.append([i+1, cx1, cy1, Vx, Vy, Ax, Ay])

df = pd.DataFrame(results, columns=['Frame', 'X', 'Y', 'Vx', 'Vy', 'Ax', 'Ay'])
window_length = min(7, len(df))

# Savitzky-Golay フィルタ
df['Vx_smooth_sg'] = savgol_filter(df['Vx'], window_length=window_length, polyorder=2)
df['Vy_smooth_sg'] = savgol_filter(df['Vy'], window_length=window_length, polyorder=2)

# バターワースフィルタ
cutoff = 2.0  # カットオフ周波数
fs = 30.0  # サンプリング周波数
df['Vx_smooth_butter'] = butter_lowpass_filter(df['Vx'], cutoff, fs)
df['Vy_smooth_butter'] = butter_lowpass_filter(df['Vy'], cutoff, fs)

# ガウシアンフィルタ
df['Vx_smooth_gaussian'] = gaussian_filter1d(df['Vx'], sigma=2)
df['Vy_smooth_gaussian'] = gaussian_filter1d(df['Vy'], sigma=2)

df.fillna(0, inplace=True)

# 平滑化後の速度から加速度を再計算
df['Ax_smooth'] = df['Vx_smooth_butter'].diff()
df['Ay_smooth'] = df['Vy_smooth_butter'].diff()
df['Ax_smooth'] = df['Ax_smooth'].fillna(0)
df['Ay_smooth'] = df['Ay_smooth'].fillna(0)
# df['Ax_smooth_gaussian'] = gaussian_filter1d(df['Ax_smooth'], sigma=40)
# df['Ay_smooth_gaussian'] = gaussian_filter1d(df['Ay_smooth'], sigma=40)
# Gaussian Filter with padding
sigma = 40  # Standard deviation for Gaussian filter
Ax_smooth_padded = np.pad(df['Ax_smooth'], (sigma, sigma), mode='reflect')
Ay_smooth_padded = np.pad(df['Ay_smooth'], (sigma, sigma), mode='reflect')

Ax_smooth_gaussian = gaussian_filter1d(Ax_smooth_padded, sigma=sigma)
Ay_smooth_gaussian = gaussian_filter1d(Ay_smooth_padded, sigma=sigma)
df['Ax_smooth_gaussian'] = Ax_smooth_gaussian[sigma:-sigma]
df['Ay_smooth_gaussian'] = Ay_smooth_gaussian[sigma:-sigma]

# Butterworth Filter
# df['Ax_smooth'] = df['Ax_smooth'].fillna(0)
# df['Ay_smooth'] = df['Ay_smooth'].fillna(0)
# N = 2  # Order of the filter
# Wn = 0.03  # Cutoff frequency
# b, a = butter(N, Wn)
# df['Ax_smooth_butter'] = filtfilt(b, a, df['Ax_smooth'])
# df['Ay_smooth_butter'] = filtfilt(b, a, df['Ay_smooth'])
# df['Ax_smooth_butter'] = butter_lowpass_filter(df['Ax_smooth'], cutoff, fs)
# df['Ay_smooth_butter'] = butter_lowpass_filter(df['Ay_smooth'], cutoff, fs)

df.fillna(0, inplace=True)

# 各フレームに加速度ベクトルを描画
for i in range(len(image_files) - 2):
    image2 = cv2.imread(image_files[i + 1])
    cx2, cy2 = df.loc[i, ['X', 'Y']]
    print(df.loc[i, ['Ax_smooth_gaussian', 'Ay_smooth_gaussian']])
    Ax, Ay = df.loc[i, ['Ax_smooth_gaussian', 'Ay_smooth_gaussian']]

    start_point = (int(cx2), int(cy2))
    end_point = (int(cx2 + Ax * 10 ** 4), int(cy2 + Ay * 10 ** 4))
    cv2.arrowedLine(image2, start_point, end_point, (0, 0, 255), 2)
    cv2.imwrite(f'./demo/output_frame_{i+1}.png', image2)

df.to_csv('centroid_based_acceleration_smoothV3.csv', index=False)
print("結果をcentroid_based_acceleration_smoothV3.csvに保存し、フレームを保存しました。")


In [ ]:
df = pd.DataFrame(df)
# df.loc[10, ['Ax_smooth', 'Ay_smooth']]
df['Ay_smooth_butter'].plot()
df['Ax_smooth_butter'].plot()

In [ ]:
df['Ay_smooth_butter'].plot()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import savgol_filter

# サンプルデータの作成
# data = {
#     'Frame': np.arange(0, 100),
#     'A_smooth': np.linspace(1, 3, 100) + 0.05 * np.sin(0.2 * np.pi * frames) + 0.05 * np.cos(0.3 * np.pi * frames),
#     'theoA': np.linspace(1, 3, 100)  # 理論値の線形補間
# }
read_csv = pd.read_csv('centroidBasedAccelerationSmoothV3.csv')
data = {
    'Frame': read_csv['Frame'],
    'A_smooth': read_csv['A_smooth'],
    'theoA': read_csv['theoA']
}
df = pd.DataFrame(data)

# データの抽出
frames = df['Frame']
A_smooth = df['A_smooth']
theoA = df['theoA']

# 平滑化手法の適用
# 移動平均
window_size = 7
A_smooth_moving_avg = A_smooth.rolling(window=window_size).mean()

# ガウシアンフィルタ
sigma = 40  # ガウシアンフィルタの標準偏差
A_smooth_gaussian = gaussian_filter1d(A_smooth, sigma=sigma)

# サビツキー・ゴレイフィルタ
window_length = 20
polyorder = 5
A_smooth_savgol = savgol_filter(A_smooth, window_length, polyorder)

# オリジナル、理論値、および平滑化されたデータのプロット
plt.figure(figsize=(12, 8))
plt.plot(frames, A_smooth, label='Original A_smooth', alpha=0.5)
plt.plot(frames, theoA, label='Theoretical A (theoA)', linestyle='--')
plt.plot(frames, A_smooth_moving_avg, label='Moving Average Smooth', linewidth=2)
plt.plot(frames, A_smooth_gaussian, label='Gaussian Smooth', linewidth=2)
plt.plot(frames, A_smooth_savgol, label='Savitzky-Golay Smooth', linewidth=2)
plt.xlabel('Frame')
plt.ylabel('Acceleration')
plt.title('Comparison of Smoothing Techniques to Theoretical Acceleration')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
!pip install ace_tools

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import savgol_filter, butter, filtfilt

# Sample data generation with a specific pattern for A_smooth around theoA
# np.random.seed(42)
# frames = np.arange(0, 1000)
# theoA = np.linspace(1, 3, 1000)
# A_smooth = theoA + 0.3 * np.sin(0.2 * np.pi * frames) + 0.1 * np.random.randn(1000)

# Create DataFrame
# df = pd.DataFrame({
#     'Frame': frames,
#     'A_smooth': A_smooth,
#     'theoA': theoA
# })

read_csv = pd.read_csv('centroidBasedAccelerationSmoothV3.csv')
df = {
    'Frame': read_csv['Frame'],
    'A_smooth': read_csv['A_smooth'],
    'theoA': read_csv['theoA']
}

# Moving Average
window_size = 25
df['A_smooth_moving_avg'] = df['A_smooth'].rolling(window=window_size).mean()

# # Gaussian Filter
# sigma = 15  # Standard deviation for Gaussian filter
# df['A_smooth_gaussian'] = gaussian_filter1d(df['A_smooth'], sigma=sigma)

# Gaussian Filter with padding
sigma = 10  # Standard deviation for Gaussian filter
A_smooth_padded = np.pad(df['A_smooth'], pad_width=sigma, mode='reflect')
A_smooth_gaussian = gaussian_filter1d(A_smooth_padded, sigma=sigma)
df['A_smooth_gaussian'] = A_smooth_gaussian[sigma:-sigma]

# Savitzky-Golay Filter
window_length = 60
polyorder = 4
df['A_smooth_savgol'] = savgol_filter(df['A_smooth'], window_length, polyorder)

# Butterworth Filter
N = 2  # Order of the filter
Wn = 0.03  # Cutoff frequency
b, a = butter(N, Wn)
df['A_smooth_butter'] = filtfilt(b, a, df['A_smooth'])

# Kalman Filter Implementation
def kalman_filter(data, Q, R):
    n = len(data)
    x = np.zeros(n)
    P = np.zeros(n)
    x[0] = data[0]
    P[0] = 1.0
    for k in range(1, n):
        # Prediction step
        x_pred = x[k-1]
        P_pred = P[k-1] + Q
        # Update step
        K = P_pred / (P_pred + R)
        x[k] = x_pred + K * (data[k] - x_pred)
        P[k] = (1 - K) * P_pred
    return x

# Set process variance (Q) and measurement variance (R)
Q = 0.001  # Process variance (Adjusted)
R = 5.0  # Measurement variance (Adjusted)

# Apply Kalman Filter
df['A_smooth_kalman'] = kalman_filter(df['A_smooth'], Q, R)

# Plot the original, theoretical, and smoothed data
plt.figure(figsize=(12, 8))
plt.plot(df['Frame'], df['A_smooth'], label='Original A_smooth', alpha=0.5)
plt.plot(df['Frame'], df['theoA'], label='Theoretical A (theoA)', linestyle='--')
plt.plot(df['Frame'], df['A_smooth_moving_avg'], label='Moving Average Smooth', linewidth=2)
plt.plot(df['Frame'], df['A_smooth_gaussian'], label='Gaussian Smooth', linewidth=2)
plt.plot(df['Frame'], df['A_smooth_savgol'], label='Savitzky-Golay Smooth', linewidth=2)
plt.plot(df['Frame'], df['A_smooth_butter'], label='Butterworth Smooth', linewidth=2)
plt.plot(df['Frame'], df['A_smooth_kalman'], label='Kalman Smooth', linewidth=2)
plt.xlabel('Frame')
plt.ylabel('Acceleration')
plt.title('Comparison of Smoothing Techniques to Theoretical Acceleration')
plt.legend()
plt.grid(True)
plt.show()

import ace_tools as tools; tools.display_dataframe_to_user(name="Optimized Smoothing Techniques Data", dataframe=df)

df.head()



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import savgol_filter, butter, filtfilt

# Sample data generation with a specific pattern for A_smooth around theoA
# np.random.seed(42)
# frames = np.arange(0, 1000)
# theoA = np.linspace(1, 3, 1000)
# A_smooth = theoA + 0.3 * np.sin(0.2 * np.pi * frames) + 0.1 * np.random.randn(1000)

# Create DataFrame
# df = pd.DataFrame({
#     'Frame': frames,
#     'A_smooth': A_smooth,
#     'theoA': theoA
# })

read_csv = pd.read_csv('centroidBasedAccelerationSmoothV3.csv')
df = {
    'Frame': read_csv['Frame'],
    'TheoAy': read_csv['TheoAy'],
    'TheoAx': read_csv['TheoAx'],
    'Ay_smooth': read_csv['Ay_smooth'],
    'Ax_smooth': read_csv['Ax_smooth']
}

# Moving Average
window_size = 25
df['Ax_smooth_moving_avg'] = df['Ax_smooth'].rolling(window=window_size).mean()
df['Ay_smooth_moving_avg'] = df['Ay_smooth'].rolling(window=window_size).mean()


# Gaussian Filter with padding
sigma = 40  # Standard deviation for Gaussian filter
Ax_smooth_padded = np.pad(df['Ax_smooth'], (sigma, sigma), mode='reflect')
Ay_smooth_padded = np.pad(df['Ay_smooth'], (sigma, sigma), mode='reflect')

Ax_smooth_gaussian = gaussian_filter1d(Ax_smooth_padded, sigma=sigma)
Ay_smooth_gaussian = gaussian_filter1d(Ay_smooth_padded, sigma=sigma)
df['Ax_smooth_gaussian'] = Ax_smooth_gaussian[sigma:-sigma]
df['Ay_smooth_gaussian'] = Ay_smooth_gaussian[sigma:-sigma]


# Savitzky-Golay Filter
window_length = 60
polyorder = 4
df['Ax_smooth_savgol'] = savgol_filter(df['Ax_smooth'], window_length, polyorder)
df['Ay_smooth_savgol'] = savgol_filter(df['Ay_smooth'], window_length, polyorder)


# Butterworth Filter
N = 2  # Order of the filter
Wn = 0.03  # Cutoff frequency
b, a = butter(N, Wn)
df['Ax_smooth_butter'] = filtfilt(b, a, df['Ax_smooth'])
df['Ay_smooth_butter'] = filtfilt(b, a, df['Ay_smooth'])


# Kalman Filter Implementation
def kalman_filter(data, Q, R):
    n = len(data)
    x = np.zeros(n)
    P = np.zeros(n)
    x[0] = data[0]
    P[0] = 1.0
    for k in range(1, n):
        # Prediction step
        x_pred = x[k-1]
        P_pred = P[k-1] + Q
        # Update step
        K = P_pred / (P_pred + R)
        x[k] = x_pred + K * (data[k] - x_pred)
        P[k] = (1 - K) * P_pred
    return x

# Set process variance (Q) and measurement variance (R)
Q = 0.001  # Process variance (Adjusted)
R = 5.0  # Measurement variance (Adjusted)

# Apply Kalman Filter
df['Ax_smooth_kalman'] = kalman_filter(df['Ax_smooth'], Q, R)
df['Ay_smooth_kalman'] = kalman_filter(df['Ay_smooth'], Q, R)


# Plot the original, theoretical, and smoothed data
plt.figure(figsize=(12, 8))
plt.plot(df['Frame'], df['Ax_smooth'], label='Original Ax_smooth', alpha=0.5)
plt.plot(df['Frame'], df['TheoAx'], label='Theoretical Ax (theoAx)', linestyle='--')
# plt.plot(df['Frame'], df['Ax_smooth_moving_avg'], label='Moving Average Smoothx', linewidth=2)
plt.plot(df['Frame'], df['Ax_smooth_gaussian'], label='Gaussian Smoothx', linewidth=2)
# plt.plot(df['Frame'], df['Ax_smooth_savgol'], label='Savitzky-Golay Smoothx', linewidth=2)
plt.plot(df['Frame'], df['Ax_smooth_butter'], label='Butterworth Smoothx', linewidth=2)
# plt.plot(df['Frame'], df['Ax_smooth_kalman'], label='Kalman Smoothx', linewidth=2)
plt.plot(df['Frame'], df['Ay_smooth'], label='Original Ay_smooth', alpha=0.5)
plt.plot(df['Frame'], df['TheoAy'], label='Theoretical Ay (theoAy)', linestyle='--')
# plt.plot(df['Frame'], df['Ay_smooth_moving_avg'], label='Moving Average Smoothy', linewidth=2)
plt.plot(df['Frame'], df['Ay_smooth_gaussian'], label='Gaussian Smoothy', linewidth=2)
# plt.plot(df['Frame'], df['Ay_smooth_savgol'], label='Savitzky-Golay Smoothy', linewidth=2)
plt.plot(df['Frame'], df['Ay_smooth_butter'], label='Butterworth Smoothy', linewidth=2)
# plt.plot(df['Frame'], df['Ay_smooth_kalman'], label='Kalman Smoothy', linewidth=2)
plt.xlabel('Frame')
plt.ylabel('Acceleration')
plt.title('Comparison of Smoothing Techniques to Theoretical Acceleration')
plt.legend()
plt.grid(True)
plt.show()

import ace_tools as tools; tools.display_dataframe_to_user(name="Optimized Smoothing Techniques Data", dataframe=df)

df.head()


In [ ]:
df = pd.DataFrame(df)
df['Ax_smooth_gaussian'].head()

In [ ]:
df['Ay_smooth_gaussian'].head()

In [ ]:
import cv2
import numpy as np
import glob
import os
import pandas as pd
from scipy.signal import savgol_filter, butter, filtfilt
from scipy.ndimage import gaussian_filter1d
from filterpy.kalman import KalmanFilter

def initialize_kalman_filter():
    kf = KalmanFilter(dim_x=2, dim_z=1)
    kf.F = np.array([[1, 1], [0, 1]])  # 状態遷移行列
    kf.H = np.array([[1, 0]])          # 観測行列
    kf.P *= 1000.                      # 誤差共分散行列の初期設定
    kf.R = np.array([[5]])             # 観測ノイズの共分散
    kf.Q = np.array([[1, 0], [0, 1]])  # プロセスノイズの共分散
    kf.x = np.array([[0], [0]])        # 初期状態
    return kf

def kalman_filter_acceleration(data):
    kf = initialize_kalman_filter()
    filtered_data = []
    for measurement in data:
        kf.predict()
        kf.update([measurement])
        filtered_data.append(kf.x[0][0])
    return np.array(filtered_data)

def segment_and_compute_centroid(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    moments = cv2.moments(binary)
    if moments['m00'] != 0:
        cx = moments['m10'] / moments['m00']
        cy = moments['m01'] / moments['m00']
    else:
        cx, cy = 0.0, 0.0
    return cx, cy

def compute_velocity_and_acceleration(centroid1, centroid2, centroid3, dt):
    cx1, cy1 = centroid1
    cx2, cy2 = centroid2
    cx3, cy3 = centroid3

    Vx1 = (cx2 - cx1) / dt
    Vy1 = (cy2 - cy1) / dt
    Vx2 = (cx3 - cx2) / dt
    Vy2 = (cy3 - cy2) / dt

    Ax = (Vx2 - Vx1) / dt
    Ay = (Vy2 - Vy1) / dt

    return Vx1, Vy1, Ax, Ay

def theoretical_values(fr, r, v):
    w = v / r
    x = r * np.cos(w * fr)
    y = r * np.sin(w * fr)
    Vx = -v * np.sin(w * fr)
    Vy = v * np.cos(w * fr)
    Ax = -v * w * np.cos(w * fr)
    Ay = -v * w * np.sin(w * fr)
    return x, y, Vx, Vy, Ax, Ay

def butter_lowpass_filter(data, cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

image_folder = 'newmethod'
image_files = sorted(glob.glob(os.path.join(image_folder, '*.png')))
dt = 1 / 30.0

results = []
for i in range(len(image_files) - 2):
    image1 = cv2.imread(image_files[i])
    image2 = cv2.imread(image_files[i + 1])
    image3 = cv2.imread(image_files[i + 2])
    cx1, cy1 = segment_and_compute_centroid(image1)
    cx2, cy2 = segment_and_compute_centroid(image2)
    cx3, cy3 = segment_and_compute_centroid(image3)
    Vx, Vy, Ax, Ay = compute_velocity_and_acceleration((cx1, cy1), (cx2, cy2), (cx3, cy3), dt)
    results.append([i+1, cx1, cy1, Vx, Vy, Ax, Ay])

df = pd.DataFrame(results, columns=['Frame', 'X', 'Y', 'Vx', 'Vy', 'Ax', 'Ay'])

# Savitzky-Golay フィルタ
window_length = min(7, len(df))
df['Vx_smooth_sg'] = savgol_filter(df['Vx'], window_length=window_length, polyorder=2)
df['Vy_smooth_sg'] = savgol_filter(df['Vy'], window_length=window_length, polyorder=2)

# バターワースフィルタ
cutoff = 2.0  # カットオフ周波数
fs = 30.0  # サンプリング周波数
df['Vx_smooth_butter'] = butter_lowpass_filter(df['Vx'], cutoff, fs)
df['Vy_smooth_butter'] = butter_lowpass_filter(df['Vy'], cutoff, fs)

# ガウシアンフィルタ
df['Vx_smooth_gaussian'] = gaussian_filter1d(df['Vx'], sigma=2)
df['Vy_smooth_gaussian'] = gaussian_filter1d(df['Vy'], sigma=2)

df.fillna(0, inplace=True)

# 平滑化後の速度から加速度を再計算
df['Ax_smooth'] = df['Vx_smooth_butter'].diff() / dt
df['Ay_smooth'] = df['Vy_smooth_butter'].diff() / dt

df['Ax_smooth'] = df['Ax_smooth'].fillna(0)
df['Ay_smooth'] = df['Ay_smooth'].fillna(0)
sigma = 40  # Standard deviation for Gaussian filter
Ax_smooth_padded = np.pad(df['Ax_smooth'], (sigma, sigma), mode='reflect')
Ay_smooth_padded = np.pad(df['Ay_smooth'], (sigma, sigma), mode='reflect')

Ax_smooth_gaussian = gaussian_filter1d(Ax_smooth_padded, sigma=sigma)
Ay_smooth_gaussian = gaussian_filter1d(Ay_smooth_padded, sigma=sigma)
df['Ax_smooth_gaussian'] = Ax_smooth_gaussian[sigma:-sigma]
df['Ay_smooth_gaussian'] = Ay_smooth_gaussian[sigma:-sigma]
# df['Ax_smooth_butter'] = butter_lowpass_filter(df['Ax_smooth'], cutoff, fs)
# df['Ay_smooth_butter'] = butter_lowpass_filter(df['Ay_smooth'], cutoff, fs)

# 動画ライターの初期化
output_video = 'output_video.avi'
frame_width = int(image2.shape[1])
frame_height = int(image2.shape[0])
out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))

# 各フレームに加速度ベクトルを描画
for i in range(len(image_files) - 2):
    image2 = cv2.imread(image_files[i + 1])
    cx2, cy2 = df.loc[i, ['X', 'Y']]
    Ax, Ay = df.loc[i, ['Ax_smooth_gaussianr', 'Ay_smooth_gaussian']]

    start_point = (int(cx2), int(cy2))
    end_point = (int(cx2 + Ax * 10), int(cy2 + Ay * 10))
    cv2.arrowedLine(image2, start_point, end_point, (0, 0, 255), 2)

    # フレームを動画に追加
    out.write(image2)

# 動画ライターを解放
out.release()

df.to_csv('centroid_based_acceleration_smoothV3.csv', index=False)
print("結果をcentroid_based_acceleration_smoothV3.csvに保存し、動画を生成しました。")


# 動画書き起こし

In [ ]:
import cv2
import glob
import os

def create_video_from_frames(image_folder, output_video, fps=30):
    # 画像ファイルを取得
    image_files = sorted(glob.glob(os.path.join(image_folder, '*.png')))

    # 最初のフレームを読み込み、フレームのサイズを取得
    first_frame = cv2.imread(image_files[0])
    frame_height, frame_width, _ = first_frame.shape

    # 動画ライターの初期化
    out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

    # 各画像を動画に追加
    for image_file in image_files:
        frame = cv2.imread(image_file)
        out.write(frame)

    # 動画ライターを解放
    out.release()
    print(f"動画ファイル {output_video} を作成しました。")

# 使用例
image_folder = 'demo'  # 画像が保存されているフォルダのパス
output_video = './output_video.avi'      # 作成する動画ファイルの名前
fps = 30                              # フレームレート

create_video_from_frames(image_folder, output_video, fps)


In [ ]:
import decimal
import math

# decimal のコンテキスト設定
decimal.getcontext().prec = 50  # 精度を50桁に設定

# 円運動のパラメータ
r = decimal.Decimal('1.0')  # 半径
omega = decimal.Decimal('1.0')  # 角速度

# 時間の変化
dt = decimal.Decimal('0.01')  # 時間の微小変化

# 速度の計算
# 速度ベクトル v(t) = (-r * omega * sin(omega * t), r * omega * cos(omega * t))
def velocity(t):
    vx = -r * omega * decimal.Decimal(math.sin(float(omega * t)))
    vy = r * omega * decimal.Decimal(math.cos(float(omega * t)))
    return (vx, vy)

# 速度ベクトルの差分を求める
t1 = decimal.Decimal('0.0')  # 時刻 t1
t2 = t1 + dt  # 時刻 t2

v1 = velocity(t1)  # 時刻 t1 の速度
v2 = velocity(t2)  # 時刻 t2 の速度

# 速度ベクトルの差分
dvx = v2[0] - v1[0]
dvy = v2[1] - v1[1]

# 加速度ベクトル (a = dv/dt)
ax = dvx / dt
ay = dvy / dt

print("速度ベクトルの差分:")
print(f"dvx: {dvx}")
print(f"dvy: {dvy}")

print("加速度ベクトル:")
print(f"ax: {ax}")
print(f"ay: {ay}")
